In [4]:
import pandas as pd
import pyodbc
import os

In [3]:
username_ = 'dlabs'
password_ = 'L@bs2020'
x = "DSN=prdsso; UID={}; PWD={}".format(username_, password_)
SSO = pyodbc.connect(x)

In [5]:
#connect to db
def datamart_connect():
    username = ''
    password = ''
    s = "DSN=PRDDM; UID={}; PWD={}".format(username, password)
    AMADM = pyodbc.connect(s)
    return AMADM

def org_manager_connect():
    username_ = 'dlabs'
    password_ = 'L@bs2020'
    x = "DSN=prdsso; UID={}; PWD={}".format(username_, password_)
    SSO = pyodbc.connect(x)
    return SSO

def get_org_query():
    query = \
        '''
        SELECT *
        FROM
        SSO.ORGANIZATION
        '''
    return query

def get_order_query(months, years):
    query= \
    f""" 
    SELECT DISTINCT 
    D.FULL_DT,
    H.MED_EDU_NBR AS ME,
    H.PARTY_ID,
    O.ORDER_NBR,
    O.ORDER_PRODUCT_ID, 
    O.ORDER_PHYSICIAN_HIST_KEY,
    O.CUSTOMER_KEY,
    D.YR,
    D.MONTH_NBR
    FROM 
    AMADM.DIM_DATE D, AMADM.FACT_EPROFILE_ORDERS O, AMADM.DIM_PHYSICIAN_HIST H
    WHERE  
    D.DATE_KEY = O.ORDER_DT_KEY
    AND 
    H.PHYSICIAN_HIST_KEY = O.ORDER_PHYSICIAN_HIST_KEY
    AND
    D.MONTH_NBR in {months}
    AND
    D.YR in {years}
    """
    return query

#define customer query
def get_customer_query():
    query = \
    """
    SELECT DISTINCT
    C.CUSTOMER_KEY, 
    C.CUSTOMER_NBR,
    C.CUSTOMER_ISELL_LOGIN,
    C.CUSTOMER_NAME,
    C.CUSTOMER_TYPE_DESC,
    C.CUSTOMER_TYPE,
    C.CUSTOMER_CATEGORY_DESC
    FROM 
    AMADM.dim_customer c  
    """
    return query        

def newest_delta(path, text):
    '''Get newest filename'''
    files = os.listdir(path)
    paths = [os.path.join(path, basename) for basename in files if text in basename]
    return max(paths, key=os.path.getctime)

def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums

def get_ppd():
    ppd_loc = 'C:/Users/vigrose/Data/PPD'
    ppd_file = newest_delta(ppd_loc, 'ppd')
    ppd = pd.read_csv(ppd_file)
    ppd['ME']=fix_me(list(ppd['ME']))
    return ppd

def get_addresses():
    query = get_org_query()
    org_addresses = pd.read_sql(con=SSO, sql=query)
    org_addresses['ADVANTAGE_ID'] = org_addresses.ADVANTAGE_ID.fillna('0')
    org_addresses = org_addresses[org_addresses.ADVANTAGE_ID!='']
    org_addresses['CUSTOMER_NBR']=[float(x) for x in org_addresses.ADVANTAGE_ID]
    org_addresses = org_addresses.sort_values('STATUS').drop_duplicates('ADVANTAGE_ID', keep='last')
    return org_addresses

def get_datamart_results(months, years, order_type='all', customer_type='all'):
    #Connect
    AMADM = datamart_connect()
    #Define Queries
    order_query = get_order_query(months, years)
    customer_query = get_customer_query()
    #Execute queries
    orders = pd.read_sql(con=AMADM, sql=order_query)
    print(len(orders))
    customers = pd.read_sql(con=AMADM, sql=customer_query)
    print(len(customers))
    addresses = get_addresses()
    #Filter
    customers.CUSTOMER_KEY = customers.CUSTOMER_KEY.astype(str)
    customers = customers.fillna('None')
    if customer_type=='all':
        class_cust = customers[customers.CUSTOMER_NAME != 'None']
    else:
        class_cust = customers[(customers.CUSTOMER_CATEGORY_DESC.isin(customer_type))&(customers.CUSTOMER_NAME != 'None')]
    if order_type == 'reapp':
        orders = orders[orders.ORDER_PRODUCT_ID.isin([4915514])]
    elif order_type == 'app':
        orders = orders[orders.ORDER_PRODUCT_ID.isin([4915513])]
    else:
        orders = orders[orders.ORDER_PRODUCT_ID.isin([4915513,4915514])]
    #Get ppd
    # ppd = get_ppd()
    #Merge and clean
    # physician_orders = pd.merge(orders, ppd, on='ME')
    print(len(orders))
    print(len(class_cust))
    results = pd.merge(orders, class_cust, on = 'CUSTOMER_KEY').drop_duplicates()
    results['CUSTOMER_NBR'] = results.CUSTOMER_NBR.astype(float)
    results = pd.merge(results, addresses, on='CUSTOMER_NBR')
    return results

In [6]:
orgs = get_addresses()

In [12]:
orgs[['ORG_ID', 'ORG_NAME', 'STATUS', 'ADVANTAGE_ID',
       'ADDRESS_LINE_1', 'ADDRESS_LINE_2', 'CITY', 'STATE', 'ZIPCODE',
       'ORGANIZATION_TYPE', 'PHONE_NUMBER', 'CUSTOMER_NBR']].to_csv('../../Data/Credentialling/Organizations_210609.csv', index=False)

In [9]:
REAPP = get_datamart_results((1,2,3,4,5,6,7,8,9,10,11,12),(2016, 2017, 2018, 2019, 2020, 2021),'reapp')

3049750
148831
491574
148227


In [18]:
APP.groupby(['YR','MONTH_NBR']).count()[:30][['FULL_DT']]

FULL_DT
YR   MONTH_NBR         
2016 1            23417
     2            25841
     3            29291
     4            30277
     5            31258
     6            32753
     7            28906
     8            31602
     9            26547
     10           25113
     11           24057
     12           23106
2017 1            24824
     2            24600
     3            31676
     4            33087
     5            34725
     6            35295
     7            30831
     8            31990
     9            28326
     10           27258
     11           24340
     12           21433
2018 1            26606
     2            24763
     3            29163
     4            30090
     5            30276
     6            29957

In [14]:
APP = get_datamart_results((1,2,3,4,5,6,7,8,9,10,11,12),(2016, 2017, 2018, 2019, 2020, 2021),'app')

3049750
148831
1754859
148227


In [25]:
ALL = pd.concat([APP, REAPP])

In [54]:
new_2_years = get_datamart_results((1,2,3,4,5,6,7,8,9,10,11,12),(2019, 2020, 2021),'app')

1254451
148529
730550
147930


In [2]:
PPD = pd.read_csv('../../Data/PPD/ppd_data_20210206.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,20,42,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [57]:
PPD[(PPD.FIRST_NAME=='ALLISON')&(PPD.LAST_NAME=='CRAWFORD')][['TELEPHONE_NUMBER', 'STATE']]

,TELEPHONE_NUMBER,STATE
202650,7.753238e+09,IL
212574,NaN,AZ


In [49]:
PPD.dtypes[40:]

GENDER                            float64
TELEPHONE_NUMBER                  float64
PRESUMED_DEAD_FLAG                 object
FAX_NUMBER                        float64
TOP_CD                              int64
PE_CD                               int64
PRIM_SPEC_CD                       object
SEC_SPEC_CD                        object
MPA_CD                             object
PRA_RECIPIENT                      object
PRA_EXP_DT                        float64
GME_CONF_FLG                       object
FROM_DT                           float64
TO_DT                             float64
YEAR_IN_PROGRAM                   float64
POST_GRADUATE_YEAR                float64
GME_SPEC_1                         object
GME_SPEC_2                         object
TRAINING_TYPE                     float64
GME_INST_STATE                    float64
GME_INST_ID                       float64
MEDSCHOOL_STATE                     int64
MEDSCHOOL_ID                        int64
MEDSCHOOL_GRAD_YEAR               

In [8]:
results

,FULL_DT,ME,PARTY_ID,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
0,2020-02-03,-1,-1,04599704,4915514,-1,52184,000002041391,48HP62497,Fort Duncan Regional Medical Center,Eprofile Webstore Customer,Eprofile,Hospital
1,2020-02-21,01601920065,2102874,42839855,4915514,7748133,52184,000002041391,48HP62497,Fort Duncan Regional Medical Center,Eprofile Webstore Customer,Eprofile,Hospital
2,2020-03-18,70422910032,2764768,43142345,4915514,9397299,52184,000002041391,48HP62497,Fort Duncan Regional Medical Center,Eprofile Webstore Customer,Eprofile,Hospital
3,2020-03-17,39621960010,2894310,43141203,4915514,9451268,52184,000002041391,48HP62497,Fort Duncan Regional Medical Center,Eprofile Webstore Customer,Eprofile,Hospital
4,2020-03-16,04201050010,3415902,43139677,4915514,9477458,52184,000002041391,48HP62497,Fort Duncan Regional Medical Center,Eprofile Webstore Customer,Eprofile,Hospital
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95750,2020-11-25,02846860501,2176302,43851817,4915514,11543260,61408,000002087324,28HP06110,Cedar County Memorial Hosp.,Eprofile Webstore Customer,Eprofile,Hospital
95751,2020-12-23,00771140262,5864127,43950311,4915514,11607274,124276,000002086635,48AM75011,Highline South Ambulatory Surgery Center,Eprofile Webstore Customer,Eprofile,Ambulatory Care
95752,2020-12-18,01875780316,2040254,43939681,4915514,11668241,121774,000002105482,38HP80652,"Ohio North East Health Systems, Inc",Eprofile Webstore Customer,Eprofile,Group
95753,2020-12-28,57503080020,5291619,43955774,4915514,11672472,57454,000002104355,36HP72846,Catawba Valley Medical Center,Eprofile Webstore Customer,Eprofile,Hospital


In [10]:
orgs = pd.read_excel('../../Data/Credentialling/Org_Addresses.xlsx')

In [11]:
orgs

,Customer Number,Company Name,Attention First,Attention Last,Street 1ST,Street 2ND,Street 3RD,City,State,Zip Code,Phone Number,Email Address
0,13323,Cary Medical Center,NaN,NaN,163 Van Buren Rd Ste 1,NaN,NaN,Caribou,ME,047363588,207 498-1227,COUELLETTE@CARYMED.ORG
1,17736,Amarillo VA Health Care System,NaN,NaN,6010 W Amarillo Blvd,NaN,NaN,Amarillo,TX,791061990,806 355-9703,NaN
2,18125,Mason District Hospital,NaN,NaN,615 N Promenade,PO Box 530,NaN,Havana,IL,626440530,309 543-8579,NaN
3,18760,Woodlawn Hospital,NaN,NaN,1400 E 9th St,NaN,NaN,Rochester,IN,469758937,574 224-1173,NaN
4,486394,"Feather River Tribal Health, Inc.",NaN,NaN,2145 5th Ave,NaN,NaN,Oroville,CA,959655870,530 534-5394,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
11407,4007512,"Advanced Surgery Center of Bethesda, LLC",NaN,NaN,6430 Rockledge Dr Ste 160,NaN,NaN,Bethesda,MD,208171847,NaN,NaN
11408,4009224,Hope Pregnancy Ministries,NaN,NaN,1281 Burns Way,NaN,NaN,Kalispell,MT,599013109,NaN,NaN
11409,4013302,Capital Orthopaedic Surgery Center,NaN,NaN,20680 Seneca Meadows Pkwy Ste 200,NaN,NaN,Germantown,MD,208767029,NaN,NaN
11410,2501871519,Blue Care Network,Felecia,Williams,28856 Lake Park Dr,NaN,NaN,Farmington Hills,MI,483312631,NaN,FWILLIAMS@BCBSM.COM


In [12]:
results.drop_duplicates('ME')

,FULL_DT,ME,PARTY_ID,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
0,2020-02-03,-1,-1,04599704,4915514,-1,52184,000002041391,48HP62497,Fort Duncan Regional Medical Center,Eprofile Webstore Customer,Eprofile,Hospital
1,2020-02-21,01601920065,2102874,42839855,4915514,7748133,52184,000002041391,48HP62497,Fort Duncan Regional Medical Center,Eprofile Webstore Customer,Eprofile,Hospital
2,2020-03-18,70422910032,2764768,43142345,4915514,9397299,52184,000002041391,48HP62497,Fort Duncan Regional Medical Center,Eprofile Webstore Customer,Eprofile,Hospital
3,2020-03-17,39621960010,2894310,43141203,4915514,9451268,52184,000002041391,48HP62497,Fort Duncan Regional Medical Center,Eprofile Webstore Customer,Eprofile,Hospital
4,2020-03-16,04201050010,3415902,43139677,4915514,9477458,52184,000002041391,48HP62497,Fort Duncan Regional Medical Center,Eprofile Webstore Customer,Eprofile,Hospital
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95747,2020-09-17,04804951526,2434286,43617220,4915514,11501616,142795,000002104565,05AM73688,"Valencia Aestetics, Inc.",Eprofile Webstore Customer,Eprofile,Ambulatory Care
95748,2020-10-28,03101810266,2348740,43796332,4915514,11524497,138371,000004009224,None,Hope Pregnancy Ministries,Eprofile Webstore Customer,Eprofile,Ambulatory Care
95751,2020-12-23,00771140262,5864127,43950311,4915514,11607274,124276,000002086635,48AM75011,Highline South Ambulatory Surgery Center,Eprofile Webstore Customer,Eprofile,Ambulatory Care
95752,2020-12-18,01875780316,2040254,43939681,4915514,11668241,121774,000002105482,38HP80652,"Ohio North East Health Systems, Inc",Eprofile Webstore Customer,Eprofile,Group


In [14]:
results.CUSTOMER_NBR = results.CUSTOMER_NBR.astype(float)
cols = {'Customer Number':'CUSTOMER_NBR',
            'Street 1ST':'addr_line_1',
            'Street 2ND':'addr_line_2',
            'Street 3RD':'addr_line_3',
            'City':'addr_city',
            'State':'addr_state',
            'Zip Code':'addr_zip'
            }
addresses = orgs.rename(columns=cols)
all_results = pd.merge(addresses, results, on='CUSTOMER_NBR')

In [15]:
all_results

,CUSTOMER_NBR,Company Name,Attention First,Attention Last,addr_line_1,addr_line_2,addr_line_3,addr_city,addr_state,addr_zip,...,PARTY_ID,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
0,18760,Woodlawn Hospital,NaN,NaN,1400 E 9th St,NaN,NaN,Rochester,IN,469758937,...,2026896,43204153,4915514,7713049,60083,17HP72434,Woodlawn Hospital,Eprofile Webstore Customer,Eprofile,Hospital
1,18760,Woodlawn Hospital,NaN,NaN,1400 E 9th St,NaN,NaN,Rochester,IN,469758937,...,2125662,42822050,4915514,7758185,60083,17HP72434,Woodlawn Hospital,Eprofile Webstore Customer,Eprofile,Hospital
2,18760,Woodlawn Hospital,NaN,NaN,1400 E 9th St,NaN,NaN,Rochester,IN,469758937,...,2279442,43727533,4915514,7892485,60083,17HP72434,Woodlawn Hospital,Eprofile Webstore Customer,Eprofile,Hospital
3,18760,Woodlawn Hospital,NaN,NaN,1400 E 9th St,NaN,NaN,Rochester,IN,469758937,...,2299688,43113481,4915514,7906269,60083,17HP72434,Woodlawn Hospital,Eprofile Webstore Customer,Eprofile,Hospital
4,18760,Woodlawn Hospital,NaN,NaN,1400 E 9th St,NaN,NaN,Rochester,IN,469758937,...,4577530,43243148,4915514,8163184,60083,17HP72434,Woodlawn Hospital,Eprofile Webstore Customer,Eprofile,Hospital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95493,3945377,Iron Horse Surgery Center,NaN,NaN,100 Park Pl Ste 100 # 110B,NaN,NaN,San Ramon,CA,945832063,...,3327278,43625500,4915514,11222196,137739,None,Iron Horse Surgery Center,Eprofile Webstore Customer,Eprofile,Ambulatory Care
95494,3945377,Iron Horse Surgery Center,NaN,NaN,100 Park Pl Ste 100 # 110B,NaN,NaN,San Ramon,CA,945832063,...,3510552,43783787,4915514,11536984,137739,None,Iron Horse Surgery Center,Eprofile Webstore Customer,Eprofile,Ambulatory Care
95495,3991990,ASC Assist LLC,NaN,NaN,63 Lake Diamond Blvd,NaN,NaN,Ocala,FL,344725006,...,2775537,43426990,4915514,10481186,138002,None,Asc Assist Llc,Eprofile Webstore Customer,Eprofile,Consulting
95496,3991990,ASC Assist LLC,NaN,NaN,63 Lake Diamond Blvd,NaN,NaN,Ocala,FL,344725006,...,2775537,43764361,4915514,10481186,138002,None,Asc Assist Llc,Eprofile Webstore Customer,Eprofile,Consulting


In [27]:
ppd = PPD[['ME', 'MAILING_NAME',
       'MAILING_LINE_1',
       'MAILING_LINE_2', 'CITY', 'STATE', 'ZIP', 'POLO_MAILING_LINE_1', 'POLO_MAILING_LINE_2',
       'POLO_CITY', 'POLO_STATE', 'POLO_ZIP']]

In [28]:
rez = all_results[['addr_line_1', 'addr_line_2', 'addr_line_3', 'addr_city', 'addr_state',
       'addr_zip', 'FULL_DT', 'ME',
       'CUSTOMER_NAME',
       'CUSTOMER_CATEGORY_DESC']]

In [23]:
ppd['ME'] = fix_me(ppd.ME)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
ZING = pd.merge(ppd, rez, on='ME')

In [34]:
count = 0
for row in ZING.itertuples():
    if row.STATE != row.addr_state:
        print('')
        print(row.CUSTOMER_CATEGORY_DESC)
        print(row.FULL_DT)
        print(row.STATE)
        print(row.POLO_STATE)
        print(row.addr_state)
        count +=1


Credit Verification Org
2020-06-02
AL
nan
SC

Ambulatory Care
2020-02-07
SC
SC
TN

Managed Care
2020-03-18
SC
SC
PA

Recruitment
2020-04-15
TX
TX
FL

Hospital
2020-12-09
AL
AL
TN

Managed Care
2020-06-10
MS
MS
NJ

Managed Care
2020-03-30
AL
AL
NJ

Hospital
2020-05-10
AL
AL
GA

Group
2020-12-21
AL
AL
GA

Hospital
2020-02-25
AL
AL
GA

Hospital
2020-10-29
GA
GA
SC

Credit Verification Org
2020-07-07
AL
AL
SC

Managed Care
2020-03-10
SC
SC
PA

Ambulatory Care
2020-07-08
CT
CT
TN

Managed Care
2020-11-12
SC
SC
PA

Hospital
2020-09-29
SC
SC
AL

Hospital
2020-11-11
TX
TX
AL

Group
2020-07-29
IL
IL
MI

Hospital
2020-07-01
IL
IL
CO

Hospital
2020-11-19
AL
AL
TN

Hospital
2020-07-06
AL
AL
MS

Hospital
2020-08-28
VA
VA
FL

Hospital
2020-10-28
AL
AL
MS

Managed Care
2020-08-05
AL
AL
GA

Ambulatory Care
2020-09-30
TN
TN
GA

Hospital
2020-12-09
AL
AL
TN

Hospital
2020-01-14
GA
GA
AL

Managed Care
2020-01-14
MA
nan
NJ

Long Term Care
2020-12-18
CO
CO
PA

Ambulatory Care
2020-09-28
LA
LA
TN

Managed 

CA
SC

Ambulatory Care
2020-10-27
NC
NC
TN

Ambulatory Care
2020-12-01
AZ
AZ
TN

Credit Verification Org
2020-11-23
MO
MO
OK

Ambulatory Care
2020-06-19
AZ
AZ
TN

Hospital
2020-08-04
AZ
AZ
AL

Hospital
2020-09-03
IA
IA
SD

Ambulatory Care
2020-09-23
AZ
AZ
TN

Credit Verification Org
2020-03-11
OR
OR
IL

Ambulatory Care
2020-06-17
AZ
AZ
TN

Hospital
2020-10-05
MN
MN
ME

Hospital
2020-06-09
MN
MN
CA

Hospital
2020-05-11
MN
MN
AL

Hospital
2020-06-29
MN
MN
VT

Hospital
2020-08-24
MN
MN
SD

Group
2020-09-03
MN
MN
WA

Hospital
2020-02-13
MN
MN
PA

Hospital
2020-03-04
MN
MN
NY

Hospital
2020-09-25
MN
MN
WA

Health Related
2020-07-28
MN
MN
NY

Hospital
2020-02-03
TN
KY
KY

Credit Verification Org
2020-08-31
AZ
AZ
SD

Managed Care
2020-01-27
AZ
AZ
NJ

Hospital
2020-02-27
CA
CA
PA

Ambulatory Care
2020-10-08
AZ
AZ
TN

Hospital
2020-11-09
HI
nan
AZ

Managed Care
2020-10-23
WI
WI
MI

Ambulatory Care
2020-07-21
AZ
AZ
TN

Hospital
2020-07-27
ID
nan
KY

Hospital
2020-03-27
ID
nan
TX

Ambulatory Care

MN
TX

Hospital
2020-03-18
FL
FL
CA

Hospital
2020-04-23
GA
nan
OR

Hospital
2020-05-21
GA
nan
CA

Ambulatory Care
2020-01-02
CA
CA
TN

Ambulatory Care
2020-06-24
FL
FL
AL

Ambulatory Care
2020-03-24
NY
NY
CA

Managed Care
2020-11-09
NV
nan
NJ

Managed Care
2020-08-06
OR
OR
NJ

Government
2020-05-13
NM
NM
AZ

Recruitment
2020-04-24
CA
nan
TX

Ambulatory Care
2020-06-24
AZ
AZ
TN

Hospital
2020-02-04
PA
PA
GA

Credit Verification Org
2020-04-20
CA
CA
TX

Managed Care
2020-09-11
SC
SC
PA

Long Term Care
2020-07-07
CA
nan
TX

Managed Care
2020-01-29
CA
CA
MT

Credit Verification Org
2020-05-05
CA
CA
TX

Ambulatory Care
2020-01-02
CA
CA
TN

Group
2020-03-01
NY
nan
CA

Hospital
2020-10-02
GA
GA
AL

Managed Care
2020-10-02
CA
CA
NJ

Ambulatory Care
2020-06-24
CA
CA
TN

Credit Verification Org
2020-10-09
CA
CA
NH

Hospital
2020-08-04
CA
CA
AL

Hospital
2020-08-11
OK
OK
WV

Hospital
2020-11-24
nan
CA
CA

Credit Verification Org
2020-05-12
CA
CA
SC

Hospital
2020-08-05
MN
MN
SD

Health Related
2

ND
MN

Hospital
2020-01-21
TX
TX
NM

Hospital
2020-10-19
TN
nan
WA

Hospital
2020-04-15
UT
UT
WY

Credit Verification Org
2020-02-17
CA
CA
SC

Consulting
2020-02-10
KY
KY
FL

Managed Care
2020-06-22
NC
NC
VA

Credit Verification Org
2020-11-30
WA
nan
TX

Hospital
2020-05-06
CA
nan
OR

Ambulatory Care
2020-10-08
MD
MD
TN

Hospital
2020-09-29
TX
TX
ND

Group
2020-07-22
CA
CA
NY

Hospital
2020-06-15
NV
nan
WA

Hospital
2020-08-05
NV
nan
MA

Hospital
2020-06-24
NM
NM
AZ

Ambulatory Care
2020-08-26
ND
ND
MN

Hospital
2020-11-18
OR
OR
CA

Group
2020-09-03
OR
OR
WA

Ambulatory Care
2020-09-09
CA
CA
TN

Hospital
2020-06-24
OR
nan
WA

Hospital
2020-03-20
CA
CA
PA

Ambulatory Care
2020-10-22
NV
NV
TN

Ambulatory Care
2020-02-03
CA
CA
TN

Credit Verification Org
2020-01-21
SC
SC
TN

Credit Verification Org
2020-07-15
IN
IN
OK

Managed Care
2020-05-21
CA
CA
MI

Managed Care
2020-11-05
CA
CA
MI

Hospital
2020-12-22
CA
CA
NJ

Government
2020-05-13
CA
CA
AZ

Ambulatory Care
2020-04-22
CA
CA
OH

Manag

SC

Hospital
2020-01-13
CA
nan
IL

Hospital
2020-03-03
CA
nan
IN

Managed Care
2020-09-04
CO
CO
NJ

Managed Care
2020-02-27
NY
NY
NJ

Hospital
2020-03-17
PA
WV
WV

Hospital
2020-05-08
TX
TX
AL

Ambulatory Care
2020-02-03
CA
CA
TN

Managed Care
2020-06-29
CA
CA
NJ

Ambulatory Care
2020-03-17
AZ
AZ
TN

Long Term Care
2020-08-13
NV
NV
PA

Hospital
2020-04-08
CA
nan
NE

Managed Care
2020-06-26
CA
CA
NJ

Hospital
2020-06-29
CA
CA
TX

Managed Care
2020-10-14
CA
CA
NJ

Group
2020-10-29
CA
CA
NY

Managed Care
2020-10-15
CA
nan
NJ

Managed Care
2020-07-17
CO
CO
NJ

Ambulatory Care
2020-12-15
CA
CA
TN

Hospital
2020-10-20
CA
CA
OR

Credit Verification Org
2020-07-16
IN
IN
VA

Ambulatory Care
2020-06-19
CA
CA
TN

Hospital
2020-01-07
NV
NV
AL

Managed Care
2020-04-20
NV
NV
SD

Hospital
2020-02-04
TX
TX
AL

Hospital
2020-08-31
CA
nan
AL

Hospital
2020-09-11
CA
nan
AL

Credit Verification Org
2020-02-27
FL
nan
MO

Hospital
2020-07-06
CA
nan
OR

Hospital
2020-08-19
CA
nan
OH

Credit Verification Org


...

2020-04-22
IN
IN
PA

Hospital
2020-09-17
OH
OH
WV

Credit Verification Org
2020-01-23
OH
OH
WV

Managed Care
2020-12-29
LA
LA
PA

Managed Care
2020-02-27
FL
FL
TX

Hospital
2020-06-04
AZ
AZ
AL

Hospital
2020-09-02
MI
MI
PA

Hospital
2020-03-17
TX
TX
AL

Credit Verification Org
2020-04-28
MD
MD
SC

Credit Verification Org
2020-04-28
MD
MD
SC

Hospital
2020-03-10
TX
TX
AL

Ambulatory Care
2020-08-05
VA
VA
CA

Hospital
2020-04-01
IN
IN
AL

Hospital
2020-06-08
VA
VA
AL

Managed Care
2020-11-06
LA
LA
PA

Ambulatory Care
2020-09-03
TX
TX
TN

Managed Care
2020-09-08
DE
DE
PA

Hospital
2020-05-14
IL
nan
NY

Hospital
2020-05-05
MI
MI
PA

Managed Care
2020-05-01
ND
nan
SD

Hospital
2020-02-03
PA
PA
WV

Managed Care
2020-09-01
LA
LA
PA

Hospital
2020-02-05
VA
VA
KY

Managed Care
2020-02-04
GA
nan
NJ

Hospital
2020-09-01
NV
NV
AL

Hospital
2020-10-27
TX
TX
AL

Credit Verification Org
2020-03-03
TX
TX
SC

Credit Verification Org
2020-04-21
TX
TX
SC

Group
2020-01-31
TX
TX
PA

Hospital
2020-09-10
TX

2020-12-21
MN
nan
TX

Credit Verification Org
2020-02-20
NJ
NJ
SC

Credit Verification Org
2020-01-09
TX
TX
SC

Managed Care
2020-03-18
NJ
PA
PA

Hospital
2020-12-30
NV
NV
AL

Managed Care
2020-07-17
TN
KY
KY

Managed Care
2020-02-19
FL
FL
PA

Hospital
2020-02-25
WV
WV
OH

Hospital
2020-11-17
TX
TX
AL

Managed Care
2020-07-30
GA
GA
PA

Hospital
2020-03-06
TX
TX
AL

Hospital
2020-06-24
TX
TX
NM

Hospital
2020-05-18
KY
KY
OH

Hospital
2020-05-14
NV
NV
AL

Managed Care
2020-04-15
MI
MI
FL

Managed Care
2020-03-03
MD
DC
PA

Hospital
2020-11-03
TX
TX
AL

Managed Care
2020-03-05
IL
WI
MI

Managed Care
2020-09-14
IL
WI
MI

Credit Verification Org
2020-11-12
CA
CA
MS

Managed Care
2020-02-11
MD
DC
PA

Hospital
2020-11-24
NJ
NJ
AL

Ambulatory Care
2020-10-29
MD
MD
TN

Recruitment
2020-08-25
MN
MN
TN

Managed Care
2020-08-31
LA
LA
PA

Hospital
2020-04-01
KS
MO
MO

Managed Care
2020-07-15
CA
CA
NJ

Hospital
2020-09-24
OK
OK
MO

Hospital
2020-10-19
TX
TX
AL

Hospital
2020-10-22
IA
IA
OH

Group
202


Hospital
2020-04-29
IL
nan
IN

Hospital
2020-03-17
IL
nan
LA

Hospital
2020-06-19
IL
nan
TX

Managed Care
2020-03-16
NY
NY
PA

Hospital
2020-10-02
OH
OH
CA

Hospital
2020-01-29
NV
nan
AL

Managed Care
2020-06-02
PA
nan
MI

Credit Verification Org
2020-01-24
OK
OK
SC

Ambulatory Care
2020-07-08
NM
NM
TN

Health Related
2020-06-04
IL
IL
GA

Hospital
2020-07-16
MD
nan
OH

Hospital
2020-02-28
TN
nan
KY

Hospital
2020-08-26
TN
TN
KY

Hospital
2020-09-18
MI
nan
VA

Group
2020-09-04
CO
nan
nan

Hospital
2020-02-13
CO
nan
RI

Hospital
2020-03-17
TX
TX
AL

Managed Care
2020-06-02
NC
nan
MI

Managed Care
2020-10-22
WI
WI
MI

Managed Care
2020-06-12
CT
CT
NY

Managed Care
2020-06-30
NY
NY
PA

Hospital
2020-09-08
MN
nan
WI

Managed Care
2020-06-02
TX
TX
UT

Group
2020-01-14
NY
NY
GA

Hospital
2020-07-20
NY
NY
GA

Hospital
2020-06-24
DE
PA
PA

Credit Verification Org
2020-04-23
PA
PA
SC

Hospital
2020-06-17
RI
nan
NY

Hospital
2020-03-10
TX
TX
AL

Long Term Care
2020-11-08
TX
TX
PA

Hospital
2020-

GA
AL

Hospital
2020-07-13
MI
MI
PA

Hospital
2020-10-13
KY
KY
IN

Credit Verification Org
2020-05-14
MD
MD
SC

Ambulatory Care
2020-04-17
MI
MI
TN

Hospital
2020-09-08
OH
OH
NJ

Hospital
2020-05-13
OH
OH
PA

Hospital
2020-06-10
OH
OH
WV

Hospital
2020-08-19
OH
OH
GA

Hospital
2020-09-01
OH
OH
WV

Hospital
2020-06-05
OH
OH
MI

Credit Verification Org
2020-04-09
OH
OH
NE

Hospital
2020-08-20
OH
OH
FL

Ambulatory Care
2020-07-29
OH
OH
NY

Credit Verification Org
2020-09-02
OH
OH
IN

Hospital
2020-12-28
KY
KY
AL

Hospital
2020-08-17
SD
nan
MI

Hospital
2020-11-24
CT
MA
AL

Managed Care
2020-10-01
TX
TX
NJ

Ambulatory Care
2020-12-23
PA
nan
VA

Hospital
2020-04-09
IN
IN
AL

Hospital
2020-03-17
TN
TN
AL

Hospital
2020-06-10
MI
MI
PA

Hospital
2020-07-02
NJ
NJ
NY

Hospital
2020-06-29
TX
TX
NM

Hospital
2020-04-22
CO
nan
OH

Hospital
2020-05-13
TX
TX
nan

Hospital
2020-10-12
PA
nan
AL

Credit Verification Org
2020-01-30
IL
IN
IN

Hospital
2020-06-09
TX
TX
ND

Hospital
2020-08-27
TX
TX
nan

Ho

WI
MI

Managed Care
2020-06-02
WI
WI
MI

Credit Verification Org
2020-08-25
VA
VA
TX

Hospital
2020-02-14
TX
TX
AL

Managed Care
2020-02-25
NV
NV
NJ

Hospital
2020-10-27
OH
OH
AL

Managed Care
2020-06-26
SC
SC
PA

Health Related
2020-03-27
RI
RI
FL

Managed Care
2020-02-10
FL
FL
TX

Hospital
2020-03-06
MO
MO
IL

Hospital
2020-03-17
MO
MO
AL

Ambulatory Care
2020-02-17
MD
MD
TN

Hospital
2020-10-08
KS
MO
MO

Ambulatory Care
2020-12-01
NY
NY
TN

Hospital
2020-08-25
NC
NC
AL

Credit Verification Org
2020-04-29
NJ
nan
VA

Managed Care
2020-07-01
CA
CA
NJ

Hospital
2020-01-27
NY
NY
WA

Credit Verification Org
2020-05-01
NJ
NJ
SC

Hospital
2020-04-03
MD
nan
NJ

Hospital
2020-08-19
MD
nan
CA

Hospital
2020-04-02
MD
nan
MS

Ambulatory Care
2020-12-30
NJ
NY
NY

Long Term Care
2020-12-22
TX
TX
PA

Managed Care
2020-12-08
DE
PA
PA

Managed Care
2020-03-12
WV
WV
VA

Credit Verification Org
2020-06-01
PA
PA
OK

Hospital
2020-03-17
MO
MO
AL

Hospital
2020-08-19
MI
MI
WI

Managed Care
2020-04-15
MD
M

In [36]:
count/len(ZING)

0.28668837991174617

In [38]:
ZING = ZING.fillna('None')

In [40]:
for row in ZING.itertuples():
    if row.POLO_ZIP == row.addr_zip:
        print(row.POLO_MAILING_LINE_2)
        print(row.addr_line_1)
        print(row.CUSTOMER_NAME)
        print('')

2500 NE NEFF RD
1835 NW Pence Lane
Central Oregon Surgery Center

2500 NE NEFF RD
1835 NW Pence Lane
Central Oregon Surgery Center

108 S FRONTAGE RD W STE 101
181 S Frontage Rd W
Vail Valley Medical Center

2400 NE NEFF RD STE A
1835 NW Pence Lane
Central Oregon Surgery Center

None
None
Specialists On Call

None
None
Matagorda General Hospital

None
None
Specialists On Call Inc

500 N KEENE ST STE 101
305 N Keene St  Suite 107
Surgery Center Of Columbia

303 N KEENE ST STE 401
305 N Keene St  Suite 107
Surgery Center Of Columbia

2076 SAINT ANTHONY AVE
2076 St. Anthony Ave.
Our Lady Of Peace Residential Hospice

1 HOSPITAL DR
305 N Keene St  Suite 107
Surgery Center Of Columbia

None
None
Specialists On Call Inc

2601 S TAMIAMI TRL
2601 S Tamiami Trl
Center For Sight

None
None
Specialists On Call Inc

None
None
Specialists On Call Inc

None
None
Matagorda General Hospital

None
None
Matagorda General Hospital

None
None
Specialists On Call Inc

None
None
Matagorda Regional Medical C

In [41]:
jan_results = get_datamart_results((1),(2021),'reapp')

42463
148477
7328
147878


In [44]:
jan_results.groupby('CUSTOMER_CATEGORY_DESC').count()

,FULL_DT,ME,PARTY_ID,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE
CUSTOMER_CATEGORY_DESC,,,,,,,,,,,,
Ambulatory Care,1552,1552,1552,1552,1552,1552,1552,1552,1552,1552,1552,1552
Consulting,56,56,56,56,56,56,56,56,56,56,56,56
Credit Verification Org,160,160,160,160,160,160,160,160,160,160,160,160
Government,2,2,2,2,2,2,2,2,2,2,2,2
Group,273,273,273,273,273,273,273,273,273,273,273,273
Health Related,264,264,264,264,264,264,264,264,264,264,264,264
Hospital,4152,4152,4152,4152,4152,4152,4152,4152,4152,4152,4152,4152
Long Term Care,115,115,115,115,115,115,115,115,115,115,115,115
Managed Care,714,714,714,714,714,714,714,714,714,714,714,714


In [45]:
jan_results

,FULL_DT,ME,PARTY_ID,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
0,2021-01-26,-1,-1,04807247,4915514,-1,131103,000002188824,03HP09532,Valleywise Health Medical Center,Eprofile Webstore Customer,Eprofile,Hospital
1,2021-01-11,00301990901,1801501,43982538,4915514,11869019,131103,000002188824,03HP09532,Valleywise Health Medical Center,Eprofile Webstore Customer,Eprofile,Hospital
2,2021-01-20,02878820927,2328590,44036764,4915514,12093799,131103,000002188824,03HP09532,Valleywise Health Medical Center,Eprofile Webstore Customer,Eprofile,Hospital
3,2021-01-11,49533720250,2789096,43982536,4915514,12440307,131103,000002188824,03HP09532,Valleywise Health Medical Center,Eprofile Webstore Customer,Eprofile,Hospital
4,2021-01-20,03506031299,2954291,44036764,4915514,12581628,131103,000002188824,03HP09532,Valleywise Health Medical Center,Eprofile Webstore Customer,Eprofile,Hospital
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7287,2021-01-23,05501060544,3469401,44041224,4915514,13258182,59363,000002100323,05AM62337,Amador Surgery Center,Eprofile Webstore Customer,Eprofile,Ambulatory Care
7288,2021-01-22,01720140493,5825205,44040737,4915514,13258862,51864,000002087813,17AM70286,Carmel Ambulatory Surgery Center,Eprofile Webstore Customer,Eprofile,Ambulatory Care
7289,2021-01-27,03806080330,3560819,44050519,4915514,13261065,60508,000002100467,38AM30741,Brecksville Surgery Center,Eprofile Webstore Customer,Eprofile,Ambulatory Care
7290,2021-01-28,01106130155,5603143,44052905,4915514,13263162,63425,000003333023,None,Manchester Memorial Hospital,Eprofile Webstore Customer,Eprofile,Hospital


In [46]:
ZING[ZING.POLO_MAILING_LINE_2=='None']

,ME,MAILING_NAME,MAILING_LINE_1,MAILING_LINE_2,CITY,STATE,ZIP,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,...,POLO_ZIP,addr_line_1,addr_line_2,addr_line_3,addr_city,addr_state,addr_zip,FULL_DT,CUSTOMER_NAME,CUSTOMER_CATEGORY_DESC
0,00102000013,MICHAEL J STANFORD MD,None,4899 REYNOLDS LN,VESTAVIA,AL,35242,None,None,None,...,None,PO Box 394,None,None,Mt Pleasant,SC,294650394,2020-06-02,"Verge Operations, Llc",Credit Verification Org
56,00102021126,WENDY D KRIEGEL MD,None,566 LAKE CAMERON DR,PIKE ROAD,AL,36064,None,None,None,...,None,PO Box 706,None,None,Evergreen,AL,364010706,2020-10-15,Evergreen Medical Center,Hospital
62,00102021517,JEREMY B SMITH MD,JT 845 UAB DEPARTMENT OF ANESTHESIOLOGY,619 19TH ST S,BIRMINGHAM,AL,35233,None,None,None,...,None,PO Box 400,1010 1st St N,None,Alabaster,AL,350078608,2020-01-14,Shelby Ambulatory Surgery Center,Ambulatory Care
109,00102041453,DAVID T CHANDLER MD,None,28 MARITIME WAY,SANTA RSA BCH,FL,32459,None,None,None,...,None,4485 Furling Ln,None,None,Destin,FL,325415331,2020-10-15,Destin Surgery Center,Ambulatory Care
113,00102041585,ADAM S GREEN MD,DEPT OF RADIOLOGY,123 SUMMER ST,WORCESTER,MA,01608,None,None,None,...,None,20 Waterview Blvd Ste 100,None,None,Parsippany,NJ,070541271,2020-01-14,One Call Care Management,Managed Care
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95126,95702780153,SLOBODAN ZEC MD,None,7324 HERITAGE CT APT 1D,FRANKFORT,IL,60423,None,None,None,...,None,1243 Rickert Dr,None,None,Naperville,IL,605400954,2020-11-10,Midwest Endoscopy Center,Ambulatory Care
95136,95702910171,SVETLANA ANIC MD,None,PO BOX 2275,ATASCADERO,CA,93423,None,None,None,...,None,24511 W Jayne Ave,PO Box 5000,None,Coalinga,CA,932105000,2020-10-27,Coalinga State Hospital,Hospital
95150,95704760010,ANETA K DIMOVA MD,None,3901 MICHAEL NEILL DR,AUSTIN,TX,78730,None,None,None,...,None,396 Broadway,None,None,Kingston,NY,124014692,2020-09-25,Health Alliance Of Hudson Valley Broadway Campus,Hospital
95151,95704760010,ANETA K DIMOVA MD,None,3901 MICHAEL NEILL DR,AUSTIN,TX,78730,None,None,None,...,None,396 Broadway,None,None,Kingston,NY,124014692,2020-10-08,Health Alliance Of Hudson Valley Broadway Campus,Hospital


In [47]:
10770/len(ZING)

0.11315402395461231

In [49]:
january = pd.merge(jan_results, ppd, on='ME')

In [51]:
january = january.fillna('None')
january[january['POLO_MAILING_LINE_2']=='None']

,FULL_DT,ME,PARTY_ID,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,...,MAILING_LINE_1,MAILING_LINE_2,CITY,STATE,ZIP,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP
12,2021-01-14,18704720099,2707301,43986618,4915514,12384666,54971,000002102276,11AM90047,Hyperbaric Services Of The Palm Beaches,...,None,1558 NW 48TH PL,BOCA RATON,FL,33431,None,None,None,None,None
65,2021-01-13,03175130330,9171071,43985537,4915514,13012333,53910,000002086989,05AM74865,Valley Physicians Surgery Center At Northridge,...,None,1333 PACIFIC TERRACE DR,LAS VEGAS,NV,89128,None,None,None,None,None
69,2021-01-07,01601770098,2010967,43973596,4915514,11878307,131836,000002083811,48HP06644,Driscoll Children'S Hospital,...,None,823 S WATER ST APT 4B,CORP CHRISTI,TX,78401,None,None,None,None,None
88,2021-01-20,00519000351,1817816,44036872,4915514,11786418,54937,000002080287,05HP06315,Los Angeles County/Usc Medical Center,...,None,4545 LA GRANADA WAY,LA CANADA FLT,CA,91011,None,None,None,None,None
92,2021-01-28,00512011454,1915393,44056779,4915514,11840361,54937,000002080287,05HP06315,Los Angeles County/Usc Medical Center,...,None,PO BOX 57,DENVER,CO,80201,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7256,2021-01-20,42201126741,5708617,44035725,4915514,13183643,57849,000002100507,21HP72729,Pathway Rehabilitation Hospital,...,None,11931 LONGFELLOW CIR,SHREVEPORT,LA,71106,None,None,None,None,None
7258,2021-01-13,01642021008,2696959,43985388,4915514,13247776,56742,000002088049,05AM71192,Oasis Surgery Center,...,None,650 FOXWOOD RD,LA CANADA FLT,CA,91011,None,None,None,None,None
7264,2021-01-14,05108150132,6167682,43987942,4915514,13224050,53807,000002100731,51HP07860,Central State Hospital,...,None,9 E CLAY ST UNIT 7,RICHMOND,VA,23219,None,None,None,None,None
7265,2021-01-27,04114140363,5830039,44050651,4915514,13228524,55307,000002084704,41HP74236,Physicians Care Surgical Hospital,...,None,103 PENNSYLVANIA AVE,SINKING SPG,PA,19608,None,None,None,None,None


In [52]:
january[january['POLO_MAILING_LINE_2']=='None'].drop_duplicates('ME')

,FULL_DT,ME,PARTY_ID,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,...,MAILING_LINE_1,MAILING_LINE_2,CITY,STATE,ZIP,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP
12,2021-01-14,18704720099,2707301,43986618,4915514,12384666,54971,000002102276,11AM90047,Hyperbaric Services Of The Palm Beaches,...,None,1558 NW 48TH PL,BOCA RATON,FL,33431,None,None,None,None,None
65,2021-01-13,03175130330,9171071,43985537,4915514,13012333,53910,000002086989,05AM74865,Valley Physicians Surgery Center At Northridge,...,None,1333 PACIFIC TERRACE DR,LAS VEGAS,NV,89128,None,None,None,None,None
69,2021-01-07,01601770098,2010967,43973596,4915514,11878307,131836,000002083811,48HP06644,Driscoll Children'S Hospital,...,None,823 S WATER ST APT 4B,CORP CHRISTI,TX,78401,None,None,None,None,None
88,2021-01-20,00519000351,1817816,44036872,4915514,11786418,54937,000002080287,05HP06315,Los Angeles County/Usc Medical Center,...,None,4545 LA GRANADA WAY,LA CANADA FLT,CA,91011,None,None,None,None,None
92,2021-01-28,00512011454,1915393,44056779,4915514,11840361,54937,000002080287,05HP06315,Los Angeles County/Usc Medical Center,...,None,PO BOX 57,DENVER,CO,80201,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7256,2021-01-20,42201126741,5708617,44035725,4915514,13183643,57849,000002100507,21HP72729,Pathway Rehabilitation Hospital,...,None,11931 LONGFELLOW CIR,SHREVEPORT,LA,71106,None,None,None,None,None
7258,2021-01-13,01642021008,2696959,43985388,4915514,13247776,56742,000002088049,05AM71192,Oasis Surgery Center,...,None,650 FOXWOOD RD,LA CANADA FLT,CA,91011,None,None,None,None,None
7264,2021-01-14,05108150132,6167682,43987942,4915514,13224050,53807,000002100731,51HP07860,Central State Hospital,...,None,9 E CLAY ST UNIT 7,RICHMOND,VA,23219,None,None,None,None,None
7265,2021-01-27,04114140363,5830039,44050651,4915514,13228524,55307,000002084704,41HP74236,Physicians Care Surgical Hospital,...,None,103 PENNSYLVANIA AVE,SINKING SPG,PA,19608,None,None,None,None,None


In [2]:
add_query = \
        '''
        SELECT DISTINCT
        A.PARTY_ID,
        A.POST_CD_ID,
        P.ADDR_1,
        P.ADDR_2,
        P.ADDR_3,
        P.CITY,
        S.SRC_STATE_CD AS STATE_CD,
        P.SRC_POST_KEY,
        P.POST_CD AS ZIP,
        A.FROM_DT,
        P.SRC_POST_KEY AS POST_KEY,
        C.PURPOSE_TYPE_DESC,
        C.PURPOSE_TYPE_ID,
        C.PURPOSE_USG_DESC
        FROM
        AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C
        WHERE
        A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
        AND
        C.PURPOSE_CAT_CD='A'
        AND
        A.POST_CD_ID = P.POST_CD_ID
        AND
        P.STATE_ID = S.STATE_ID
        AND
        A.THRU_DT IS NULL
        '''
AMAEDW = pyodbc.connect(w)
usages = pd.read_sql(con=AMAEDW, sql=add_query)





In [31]:
birth_query = \
        '''
        SELECT 
        CONVERT('datetime', P.BIRTH_DT),
        P.BIRTH_DT,
        P.DEATH_DT,
        P.MED_EDU_NBR
        FROM
        AMAEDW.PERSON P
        WHERE
        P.DEATH_DT IS NOT NULL
        '''

In [3]:
username = 'vigrose'
password_edw = 'Gryffindor~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)
usages = pd.read_sql(con=AMAEDW, sql=add_query)

In [113]:
new_2_years.CUSTOMER_NBR = new_2_years.CUSTOMER_NBR.astype(float)
new_results = pd.merge(orgo, new_2_years, on='CUSTOMER_NBR')

In [116]:
orgo.groupby('STATUS').count()

,ORG_ID,ORG_NAME,PARTY_ID,ADVANTAGE_ID,ADDRESS_LINE_1,ADDRESS_LINE_2,CITY,STATE,ZIPCODE,ORGANIZATION_TYPE,PHONE_NUMBER,CUSTOMER_NBR
STATUS,,,,,,,,,,,,
0,5477,5477,0,5477,5153,2650,5091,4893,5012,4457,114,5477
1,17345,17345,0,17345,17343,17235,17343,17343,17343,17345,12290,17345


In [115]:
orgo = orgo[['ORG_ID', 'ORG_NAME', 'STATUS', 'ADVANTAGE_ID',
       'ADDRESS_LINE_1', 'ADDRESS_LINE_2', 'CITY', 'STATE', 'ZIPCODE',
       'ORGANIZATION_TYPE', 'PHONE_NUMBER', 'CUSTOMER_NBR']]

Index(['ORG_ID', 'ORG_NAME', 'PARTY_ID', 'STATUS', 'ADVANTAGE_ID',
       'ADDRESS_LINE_1', 'ADDRESS_LINE_2', 'CITY', 'STATE', 'ZIPCODE',
       'ORGANIZATION_TYPE', 'PHONE_NUMBER', 'CUSTOMER_NBR'],
      dtype='object')

In [112]:
NEW = new_results[['ORG_ID', 'ORG_NAME', 'PARTY_ID', 'STATUS', 'ADVANTAGE_ID',
       'ADDRESS_LINE_1', 'ADDRESS_LINE_2', 'CITY', 'STATE', 'ZIPCODE',
       'ORGANIZATION_TYPE', 'PHONE_NUMBER', 'CUSTOMER_NBR', 'FULL_DT', 'ME', 'PARTY_ID',
       'CUSTOMER_NAME',
       'CUSTOMER_CATEGORY_DESC']]
NEW_2 = pd.merge(ppd, NEW, on='ME')

KeyError: "['PARTY_ID'] not in index"

In [67]:
NEW_2

,ME,MAILING_NAME,MAILING_LINE_1,MAILING_LINE_2,CITY,STATE,ZIP,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,...,addr_line_1,addr_line_2,addr_line_3,addr_city,addr_state,addr_zip,FULL_DT,PARTY_ID,CUSTOMER_NAME,CUSTOMER_CATEGORY_DESC
0,00102000030,MARK T JACKSON MD,NaN,1000 BLYTHE BLVD,CHARLOTTE,NC,28203,MOUNTAIN AREA PEDIATRIC ASSOC,500 CENTREPARK DR,ASHEVILLE,...,500 W Main St # 3-515,NaN,NaN,Louisville,KY,402022946,2020-02-18,1798000,Humana Military Healthcare Services,Managed Care
1,00102000056,RAJIV B GALA MD,NaN,44 WREN ST,NEW ORLEANS,LA,70124,OCHSNER MEDICAL CENTER,4429 CLARA ST STE 400,NEW ORLEANS,...,Department of Medical Staff Services,1516 Jefferson Hwy,NaN,New Orleans,LA,701212429,2019-02-25,1888535,Ochsner Health System,Hospital
2,00102000072,JAMES H BANKSTON JR MD,NaN,6408 LAKE VISTA CIR,TUSCALOOSA,AL,35406,NaN,208 MCFARLAND CIR N STE 100,TUSCALOOSA,...,300 Med Park Dr,NaN,NaN,Thomasville,AL,367845760,2020-02-06,1844373,Thomasville Regional Medical Center,Hospital
3,00102000081,ANDREA B MCMURPHY MD,NaN,400 W DELWOOD DR,MOBILE,AL,36606,NaN,2880 DAUPHIN ST,MOBILE,...,Mastin Bldg - 6th Flr.,2451 University Hospital Dr,NaN,Mobile,AL,366172300,2019-04-15,1891432,University Of South Alabama Hospitals,Hospital
4,00102000081,ANDREA B MCMURPHY MD,NaN,400 W DELWOOD DR,MOBILE,AL,36606,NaN,2880 DAUPHIN ST,MOBILE,...,1929 Texas Ave S,NaN,NaN,College Station,TX,778403913,2019-07-25,1891432,Ama Health Solutions Business,Credit Verification Org
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718164,96501960036,KAMISHELE MAILA MD,ST RITA'S MEDICAL CENTER,730 W MARKET ST,LIMA,OH,45801,ST RITA'S MEDICAL CENTER,730 W MARKET ST,LIMA,...,Medical Staff Services - Ssc,1702 University Dr S,NaN,Fargo,ND,581034940,2020-12-08,3526200,Essentia Health,Group
718165,96501960044,MWIZA GAUSI MD,NaN,4705 JEFFERSON WOOD CT,GREENSBORO,NC,27410,DANVILLE REGIONAL MEDICAL,142 S MAIN ST,DANVILLE,...,3000 New Bern Ave,PO Box 14465,NaN,Raleigh,NC,276204465,2019-01-24,3572171,Wakemed,Hospital
718166,96501960044,MWIZA GAUSI MD,NaN,4705 JEFFERSON WOOD CT,GREENSBORO,NC,27410,DANVILLE REGIONAL MEDICAL,142 S MAIN ST,DANVILLE,...,2085 Frontis Plaza Blvd,NaN,NaN,Winston Salem,NC,271035614,2019-09-12,3572171,Novant Health,Hospital
718167,96501960044,MWIZA GAUSI MD,NaN,4705 JEFFERSON WOOD CT,GREENSBORO,NC,27410,DANVILLE REGIONAL MEDICAL,142 S MAIN ST,DANVILLE,...,500 Lauchwood Dr,NaN,NaN,Laurinburg,NC,283525599,2019-05-21,3572171,Scotland Memorial Hospital,Hospital


In [63]:
len(pd.unique(NEW_2.ME))

338118

Index(['CUSTOMER_NBR', 'Company Name', 'Attention First', 'Attention Last',
       'addr_line_1', 'addr_line_2', 'addr_line_3', 'addr_city', 'addr_state',
       'addr_zip', 'Phone Number', 'Email Address', 'FULL_DT', 'ME',
       'PARTY_ID', 'ORDER_NBR', 'ORDER_PRODUCT_ID', 'ORDER_PHYSICIAN_HIST_KEY',
       'CUSTOMER_KEY', 'CUSTOMER_ISELL_LOGIN', 'CUSTOMER_NAME',
       'CUSTOMER_TYPE_DESC', 'CUSTOMER_TYPE', 'CUSTOMER_CATEGORY_DESC'],
      dtype='object')

In [69]:
orgs

,Customer Number,Company Name,Attention First,Attention Last,Street 1ST,Street 2ND,Street 3RD,City,State,Zip Code,Phone Number,Email Address
0,13323,Cary Medical Center,NaN,NaN,163 Van Buren Rd Ste 1,NaN,NaN,Caribou,ME,047363588,207 498-1227,COUELLETTE@CARYMED.ORG
1,17736,Amarillo VA Health Care System,NaN,NaN,6010 W Amarillo Blvd,NaN,NaN,Amarillo,TX,791061990,806 355-9703,NaN
2,18125,Mason District Hospital,NaN,NaN,615 N Promenade,PO Box 530,NaN,Havana,IL,626440530,309 543-8579,NaN
3,18760,Woodlawn Hospital,NaN,NaN,1400 E 9th St,NaN,NaN,Rochester,IN,469758937,574 224-1173,NaN
4,486394,"Feather River Tribal Health, Inc.",NaN,NaN,2145 5th Ave,NaN,NaN,Oroville,CA,959655870,530 534-5394,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
11407,4007512,"Advanced Surgery Center of Bethesda, LLC",NaN,NaN,6430 Rockledge Dr Ste 160,NaN,NaN,Bethesda,MD,208171847,NaN,NaN
11408,4009224,Hope Pregnancy Ministries,NaN,NaN,1281 Burns Way,NaN,NaN,Kalispell,MT,599013109,NaN,NaN
11409,4013302,Capital Orthopaedic Surgery Center,NaN,NaN,20680 Seneca Meadows Pkwy Ste 200,NaN,NaN,Germantown,MD,208767029,NaN,NaN
11410,2501871519,Blue Care Network,Felecia,Williams,28856 Lake Park Dr,NaN,NaN,Farmington Hills,MI,483312631,NaN,FWILLIAMS@BCBSM.COM


In [28]:
username_ = 'dlabs'
password_ = 'L@bs2020'
x = "DSN=prdsso; UID={}; PWD={}".format(username_, password_)
SSO = pyodbc.connect(x)

In [29]:
username_ = 'dlabs'
password_ = 'L@bs2020'
x = "DSN=prdsso; UID={}; PWD={}".format(username_, password_)
SSO = pyodbc.connect(x)
query = \
        '''
        SELECT *
        FROM
        SSO.ORGANIZATION
        '''
ORGS_4REAL = pd.read_sql(con=SSO, sql=query)
ORGS_4REAL['ADVANTAGE_ID'] = ORGS_4REAL.ADVANTAGE_ID.fillna('0')
orgo = ORGS_4REAL[ORGS_4REAL.ADVANTAGE_ID!='']
orgo['CUSTOMER_NBR']=[float(x) for x in orgo.ADVANTAGE_ID]
orga = orgo.sort_values('STATUS').drop_duplicates('ADVANTAGE_ID', keep='last')
ALL.CUSTOMER_NBR = ALL.CUSTOMER_NBR.astype(float)
ALL_ADD = pd.merge(ALL, orga, on='CUSTOMER_NBR')

In [6]:
ORGS_4REAL

,ORG_ID,ORG_NAME,PARTY_ID,STATUS,ADVANTAGE_ID,ADDRESS_LINE_1,ADDRESS_LINE_2,CITY,STATE,ZIPCODE,ORGANIZATION_TYPE,PHONE_NUMBER
0,1,First organization test SG,None,1,000002076258,515 N State St,,Chicago,IL,606544854,HR ...,None
1,21,Molina Healthcare,None,1,000002079513,5709 W Sunset Hwy Ste 200,,Spokane,WA,992249442,HP ...,5093211320
2,22,Kindred Healthcare,None,1,000001921001,10841 White Oak Ave.,,Rancho Cucamonga,CA,91730,HP ...,7138842232
3,23,St. Jude Heritage/St. Joseph Heritage,None,1,000002079528,1515 E Orangewood Ave,,Anaheim,CA,928056824,HP ...,7149376240
4,24,Piedmont Healthcare,None,1,000002079529,2727 Paces Ferry Rd SE,,Atlanta,GA,303394053,HP ...,7708012739
...,...,...,...,...,...,...,...,...,...,...,...,...
22821,24174,LSU School of Dentistry,None,1,000004252786,Box 18,1100 Florida Ave,New Orleans,LA,701192715,CB ...,None
22822,24175,Federal Correctional Institute,None,0,None,4205 Highway 66 West,None,El Reno,OK,73036,GOV ...,None
22823,24176,Retina Consultants,None,1,000004252673,101 Plain St Ste 1,,Providence,RI,029034829,AM ...,None
22824,24177,Convergence Telemedicine Partners,None,1,000004252778,4700 Ashwood Dr Ste 200,,Blue Ash,OH,452412424,LT ...,None


In [32]:
ORGS_4REAL['ADVANTAGE_ID'] = ORGS_4REAL.ADVANTAGE_ID.fillna('0')

In [33]:
orgo = ORGS_4REAL[ORGS_4REAL.ADVANTAGE_ID!='']

In [34]:
ORGS_4REAL['ADVANTAGE_ID'] = ORGS_4REAL.ADVANTAGE_ID.fillna('0')
orgo = ORGS_4REAL[ORGS_4REAL.ADVANTAGE_ID!='']
orgo['CUSTOMER_NBR']=[float(x) for x in orgo.ADVANTAGE_ID]
orga = orgo.sort_values('STATUS').drop_duplicates('ADVANTAGE_ID', keep='last')
ALL.CUSTOMER_NBR = ALL.CUSTOMER_NBR.astype(float)
ALL_ADD = pd.merge(ALL, orga, on='CUSTOMER_NBR')


C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
orga = orgo.sort_values('STATUS').drop_duplicates('ADVANTAGE_ID', keep='last')

In [94]:
ORGS_4REAL.groupby('ADVANTAGE_ID').count().sort_values('ORG_ID', ascending=False)

,ORG_ID,ORG_NAME,PARTY_ID,STATUS,ADDRESS_LINE_1,ADDRESS_LINE_2,CITY,STATE,ZIPCODE,ORGANIZATION_TYPE,PHONE_NUMBER
ADVANTAGE_ID,,,,,,,,,,,
0,5523,5523,0,5523,5197,2587,5135,4937,5056,4510,0
000002113282,5,5,0,5,5,5,5,5,5,5,0
000002102649,4,4,0,4,4,4,4,4,4,4,4
000002099574,3,3,0,3,3,3,3,3,3,3,3
000002172007,3,3,0,3,3,3,3,3,3,1,0
...,...,...,...,...,...,...,...,...,...,...,...
000002099462,1,1,0,1,1,1,1,1,1,1,1
000002099463,1,1,0,1,1,1,1,1,1,1,1
000002099464,1,1,0,1,1,1,1,1,1,1,1


In [81]:
new_2_years

,FULL_DT,ME,PARTY_ID,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
0,2019-01-10,-1,-1,04379063,4915513,-1,60433,2101497.0,48AM73703,Westlake Dermatology,Eprofile Webstore Customer,Eprofile,Ambulatory Care
1,2019-01-09,04814840935,2530103,40564798,4915513,5494863,60433,2101497.0,48AM73703,Westlake Dermatology,Eprofile Webstore Customer,Eprofile,Ambulatory Care
2,2019-01-09,04802090558,3592237,40564798,4915513,6022790,60433,2101497.0,48AM73703,Westlake Dermatology,Eprofile Webstore Customer,Eprofile,Ambulatory Care
3,2019-01-11,-1,-1,04379859,4915513,-1,61794,2100814.0,05HP07749,Kaiser Foundation Hospital,Eprofile Webstore Customer,Eprofile,Hospital
4,2019-08-01,01643960270,1952382,41941203,4915513,5228627,61794,2100814.0,05HP07749,Kaiser Foundation Hospital,Eprofile Webstore Customer,Eprofile,Hospital
...,...,...,...,...,...,...,...,...,...,...,...,...,...
727809,2021-02-16,01720091310,3596507,44094407,4915513,13225327,146583,2085202.0,17HP63329,Indiana Orthopaedic Hospital,Eprofile Webstore Customer,Eprofile,Hospital
727810,2021-02-16,05104120409,5131120,44094848,4915513,13226963,148643,2083538.0,35HP63381,Ellis Medicine,Eprofile Webstore Customer,Eprofile,Hospital
727811,2021-02-03,03607140331,5836002,44071643,4915513,13228559,57553,2086501.0,31AM71980,Eye Surgery Center Of Northern Nevada,Eprofile Webstore Customer,Eprofile,Ambulatory Care
727812,2021-02-04,02301181321,7782206,44072707,4915513,13253354,53140,2194719.0,None,"Intellicentrics, Inc.",Eprofile Webstore Customer,Eprofile,Managed Care


In [82]:
orgs

,Customer Number,Company Name,Attention First,Attention Last,Street 1ST,Street 2ND,Street 3RD,City,State,Zip Code,Phone Number,Email Address
0,13323,Cary Medical Center,NaN,NaN,163 Van Buren Rd Ste 1,NaN,NaN,Caribou,ME,047363588,207 498-1227,COUELLETTE@CARYMED.ORG
1,17736,Amarillo VA Health Care System,NaN,NaN,6010 W Amarillo Blvd,NaN,NaN,Amarillo,TX,791061990,806 355-9703,NaN
2,18125,Mason District Hospital,NaN,NaN,615 N Promenade,PO Box 530,NaN,Havana,IL,626440530,309 543-8579,NaN
3,18760,Woodlawn Hospital,NaN,NaN,1400 E 9th St,NaN,NaN,Rochester,IN,469758937,574 224-1173,NaN
4,486394,"Feather River Tribal Health, Inc.",NaN,NaN,2145 5th Ave,NaN,NaN,Oroville,CA,959655870,530 534-5394,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
11407,4007512,"Advanced Surgery Center of Bethesda, LLC",NaN,NaN,6430 Rockledge Dr Ste 160,NaN,NaN,Bethesda,MD,208171847,NaN,NaN
11408,4009224,Hope Pregnancy Ministries,NaN,NaN,1281 Burns Way,NaN,NaN,Kalispell,MT,599013109,NaN,NaN
11409,4013302,Capital Orthopaedic Surgery Center,NaN,NaN,20680 Seneca Meadows Pkwy Ste 200,NaN,NaN,Germantown,MD,208767029,NaN,NaN
11410,2501871519,Blue Care Network,Felecia,Williams,28856 Lake Park Dr,NaN,NaN,Farmington Hills,MI,483312631,NaN,FWILLIAMS@BCBSM.COM


In [103]:
new_results

,ORG_ID,ORG_NAME,PARTY_ID_x,STATUS,ADVANTAGE_ID,ADDRESS_LINE_1,ADDRESS_LINE_2,CITY,STATE,ZIPCODE,...,PARTY_ID_y,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
0,21,Molina Healthcare,None,1,000002079513,5709 W Sunset Hwy Ste 200,,Spokane,WA,992249442,...,3584748,42683375,4915513,772798,51995,54HR60068,Molina Healthcare,Eprofile Webstore Customer,Eprofile,Hospital
1,21,Molina Healthcare,None,1,000002079513,5709 W Sunset Hwy Ste 200,,Spokane,WA,992249442,...,1981695,43302330,4915513,1394592,51995,54HR60068,Molina Healthcare,Eprofile Webstore Customer,Eprofile,Hospital
2,21,Molina Healthcare,None,1,000002079513,5709 W Sunset Hwy Ste 200,,Spokane,WA,992249442,...,1845490,40652383,4915513,5019792,51995,54HR60068,Molina Healthcare,Eprofile Webstore Customer,Eprofile,Hospital
3,21,Molina Healthcare,None,1,000002079513,5709 W Sunset Hwy Ste 200,,Spokane,WA,992249442,...,1826667,41663320,4915513,5038983,51995,54HR60068,Molina Healthcare,Eprofile Webstore Customer,Eprofile,Hospital
4,21,Molina Healthcare,None,1,000002079513,5709 W Sunset Hwy Ste 200,,Spokane,WA,992249442,...,1872142,42073612,4915513,5040765,51995,54HR60068,Molina Healthcare,Eprofile Webstore Customer,Eprofile,Hospital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732260,24147,Unitypoint Health,None,1,000004243159,1776 W Lakes Pkwy,,West Des Moines,IA,502668377,...,7784170,44081657,4915513,13001501,148377,None,Unitypoint Health,Eprofile Webstore Customer,Eprofile,Hospital
732261,24147,Unitypoint Health,None,1,000004243159,1776 W Lakes Pkwy,,West Des Moines,IA,502668377,...,11168038,44092542,4915513,13064509,148377,None,Unitypoint Health,Eprofile Webstore Customer,Eprofile,Hospital
732262,24147,Unitypoint Health,None,1,000004243159,1776 W Lakes Pkwy,,West Des Moines,IA,502668377,...,2021177,44093473,4915513,13234558,148377,None,Unitypoint Health,Eprofile Webstore Customer,Eprofile,Hospital
732263,24147,Unitypoint Health,None,1,000004243159,1776 W Lakes Pkwy,,West Des Moines,IA,502668377,...,5607016,44081657,4915513,13320038,148377,None,Unitypoint Health,Eprofile Webstore Customer,Eprofile,Hospital


In [104]:
len(new_2_years)

727814

In [108]:
usages

,PARTY_ID,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_CD,SRC_POST_KEY,ZIP,FROM_DT,POST_KEY,PURPOSE_TYPE_DESC,PURPOSE_TYPE_ID,PURPOSE_USG_DESC
0,1798316,16267637,590 Davis St E,,,Elba,AL,713596.0,36323,2005-12-16 20:20:34.847904,713596.0,Home,269,Preferred Professional Mailing Address
1,1903485,15013274,204 W Cherokee Ave,,,Cartersville,GA,713617.0,30120,2005-12-16 20:20:34.848723,713617.0,Home,269,Preferred Professional Mailing Address
2,1849882,16269589,2603 University Blvd,,,Tuscaloosa,AL,713677.0,35401,2005-12-16 20:20:34.850978,713677.0,Office,317,Preferred Office
3,1798049,16090212,412 2nd St N,,,Oneonta,AL,713741.0,35121,2005-12-16 20:20:34.853355,713741.0,Home,269,Preferred Professional Mailing Address
4,1843314,16267642,633 S Union Ave,,,Ozark,AL,713743.0,36360,2005-12-16 20:20:34.853611,713743.0,Office,317,Preferred Office
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13735203,2561036,32062011,STE 1,8 MEMORIAL MEDICAL CT,GREENVILLE PATHOLOGY ASSOCIATE,GREENVILLE,SC,25692259.0,29605,2021-02-16 23:30:24.000000,25692259.0,License - Georgia,421,No Usage
13735204,1932244,32062012,STE 110,4233 PLEASANT HILL RD,None,DULUTH,GA,25692260.0,30096,2021-02-16 23:30:24.000000,25692260.0,License - Georgia,421,No Usage
13735205,2646266,32062013,146 DAYTON R 50 LEROY STREET,None,None,POTSDAM,NY,25692261.0,13676,2021-02-16 23:30:24.000000,25692261.0,License - Florida,420,No Usage
13735206,2634159,32062014,1111 HILLCREST CT,None,None,EUSTIS,FL,25692262.0,32726,2021-02-16 23:30:24.000000,25692262.0,License - Florida,420,No Usage


In [109]:
new_2_years

,FULL_DT,ME,PARTY_ID,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
0,2019-01-10,-1,-1,04379063,4915513,-1,60433,2101497.0,48AM73703,Westlake Dermatology,Eprofile Webstore Customer,Eprofile,Ambulatory Care
1,2019-01-09,04814840935,2530103,40564798,4915513,5494863,60433,2101497.0,48AM73703,Westlake Dermatology,Eprofile Webstore Customer,Eprofile,Ambulatory Care
2,2019-01-09,04802090558,3592237,40564798,4915513,6022790,60433,2101497.0,48AM73703,Westlake Dermatology,Eprofile Webstore Customer,Eprofile,Ambulatory Care
3,2019-01-11,-1,-1,04379859,4915513,-1,61794,2100814.0,05HP07749,Kaiser Foundation Hospital,Eprofile Webstore Customer,Eprofile,Hospital
4,2019-08-01,01643960270,1952382,41941203,4915513,5228627,61794,2100814.0,05HP07749,Kaiser Foundation Hospital,Eprofile Webstore Customer,Eprofile,Hospital
...,...,...,...,...,...,...,...,...,...,...,...,...,...
727809,2021-02-16,01720091310,3596507,44094407,4915513,13225327,146583,2085202.0,17HP63329,Indiana Orthopaedic Hospital,Eprofile Webstore Customer,Eprofile,Hospital
727810,2021-02-16,05104120409,5131120,44094848,4915513,13226963,148643,2083538.0,35HP63381,Ellis Medicine,Eprofile Webstore Customer,Eprofile,Hospital
727811,2021-02-03,03607140331,5836002,44071643,4915513,13228559,57553,2086501.0,31AM71980,Eye Surgery Center Of Northern Nevada,Eprofile Webstore Customer,Eprofile,Ambulatory Care
727812,2021-02-04,02301181321,7782206,44072707,4915513,13253354,53140,2194719.0,None,"Intellicentrics, Inc.",Eprofile Webstore Customer,Eprofile,Managed Care


In [118]:
PPD.columns

Index(['ME', 'RECORD_ID', 'UPDATE_TYPE', 'ADDRESS_TYPE', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       'MAILING_LINE_2', 'CITY', 'STATE', 'ZIP', 'SECTOR', 'CARRIER_ROUTE',
       'ADDRESS_UNDELIVERABLE_FLAG', 'FIPS_COUNTY', 'FIPS_STATE',
       'PRINTER_CONTROL_CODE', 'PC_ZIP', 'PC_SECTOR', 'DELIVERY_POINT_CODE',
       'CHECK_DIGIT', 'PRINTER_CONTROL_CODE_2', 'REGION', 'DIVISION', 'GROUP',
       'TRACT', 'SUFFIX_CENSUS', 'BLOCK_GROUP', 'MSA_POPULATION_SIZE',
       'MICRO_METRO_IND', 'CBSA', 'CBSA_DIV_IND', 'MD_DO_CODE', 'BIRTH_YEAR',
       'BIRTH_CITY', 'BIRTH_STATE', 'BIRTH_COUNTRY', 'GENDER',
       'TELEPHONE_NUMBER', 'PRESUMED_DEAD_FLAG', 'FAX_NUMBER', 'TOP_CD',
       'PE_CD', 'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MPA_CD', 'PRA_RECIPIENT',
       'PRA_EXP_DT', 'GME_CONF_FLG', 'FROM_DT', 'TO_DT', 'YEAR_IN_PROGRAM',
       'POST_GRADUATE_YEAR', 'GME_SPEC_1', 'GME_SPEC_2', 'TRAINING_TYPE',
       'GME_INST_STATE', 'GME_INST_ID', 'ME

In [14]:
orgo

,ORG_ID,ORG_NAME,PARTY_ID,STATUS,ADVANTAGE_ID,ADDRESS_LINE_1,ADDRESS_LINE_2,CITY,STATE,ZIPCODE,ORGANIZATION_TYPE,PHONE_NUMBER,CUSTOMER_NBR
0,1,First organization test SG,None,1,000002076258,515 N State St,,Chicago,IL,606544854,HR ...,None,2076258.0
1,21,Molina Healthcare,None,1,000002079513,5709 W Sunset Hwy Ste 200,,Spokane,WA,992249442,HP ...,5093211320,2079513.0
2,22,Kindred Healthcare,None,1,000001921001,10841 White Oak Ave.,,Rancho Cucamonga,CA,91730,HP ...,7138842232,1921001.0
3,23,St. Jude Heritage/St. Joseph Heritage,None,1,000002079528,1515 E Orangewood Ave,,Anaheim,CA,928056824,HP ...,7149376240,2079528.0
4,24,Piedmont Healthcare,None,1,000002079529,2727 Paces Ferry Rd SE,,Atlanta,GA,303394053,HP ...,7708012739,2079529.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22821,24174,LSU School of Dentistry,None,1,000004252786,Box 18,1100 Florida Ave,New Orleans,LA,701192715,CB ...,None,4252786.0
22822,24175,Federal Correctional Institute,None,0,0,4205 Highway 66 West,None,El Reno,OK,73036,GOV ...,None,0.0
22823,24176,Retina Consultants,None,1,000004252673,101 Plain St Ste 1,,Providence,RI,029034829,AM ...,None,4252673.0
22824,24177,Convergence Telemedicine Partners,None,1,000004252778,4700 Ashwood Dr Ste 200,,Blue Ash,OH,452412424,LT ...,None,4252778.0


In [20]:
orga = orga[['ORG_ID', 'ORG_NAME', 'ADDRESS_LINE_1', 'ADDRESS_LINE_2', 'CITY', 'STATE', 'ZIPCODE',
       'ORGANIZATION_TYPE', 'PHONE_NUMBER', 'CUSTOMER_NBR']]

In [22]:
orga.to_csv('../../Data/Credentialling/Organization_Addresses_210219.csv', index=False)

In [4]:
usages

,PARTY_ID,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_CD,SRC_POST_KEY,ZIP,FROM_DT,POST_KEY,PURPOSE_TYPE_DESC,PURPOSE_TYPE_ID,PURPOSE_USG_DESC
0,1798316,16267637,590 Davis St E,,,Elba,AL,713596.0,36323,2005-12-16 20:20:34.847904,713596.0,Home,269,Preferred Professional Mailing Address
1,1903485,15013274,204 W Cherokee Ave,,,Cartersville,GA,713617.0,30120,2005-12-16 20:20:34.848723,713617.0,Home,269,Preferred Professional Mailing Address
2,1849882,16269589,2603 University Blvd,,,Tuscaloosa,AL,713677.0,35401,2005-12-16 20:20:34.850978,713677.0,Office,317,Preferred Office
3,1798049,16090212,412 2nd St N,,,Oneonta,AL,713741.0,35121,2005-12-16 20:20:34.853355,713741.0,Home,269,Preferred Professional Mailing Address
4,1843314,16267642,633 S Union Ave,,,Ozark,AL,713743.0,36360,2005-12-16 20:20:34.853611,713743.0,Office,317,Preferred Office
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13743116,15977897,32070486,10314 Solar Way,,,Charlotte,NC,25700109.0,28278,2021-02-21 20:45:40.480881,25700109.0,Unknown,466,No Usage
13743117,15977897,32070486,10314 Solar Way,,,Charlotte,NC,25700109.0,28278,2021-02-21 20:45:40.480947,25700109.0,Unknown,513,Billing Address. ...
13743118,15797429,32070487,2186 Nc 55 E,,,Lillington,NC,25700112.0,27546,2021-02-21 20:45:40.481020,25700112.0,Home,405,No Usage
13743119,15797429,32070487,2186 Nc 55 E,,,Lillington,NC,25700112.0,27546,2021-02-21 20:45:40.481104,25700112.0,Home,498,AMC Address. ...


In [36]:
orga

,ORG_ID,ORG_NAME,PARTY_ID,STATUS,ADVANTAGE_ID,ADDRESS_LINE_1,ADDRESS_LINE_2,CITY,STATE,ZIPCODE,ORGANIZATION_TYPE,PHONE_NUMBER,CUSTOMER_NBR
3829,4030,Frisco Ambulatory Surgery Center,None,0,000002086554,5616 Warren Pkwy,,Frisco,TX,750344166,AM ...,9727124800,2086554.0
3901,4105,"Neurocall, Inc",None,0,000002086636,11760 SW 40th St Ste 301,,Miami,FL,331753589,AM ...,3052204611,2086636.0
16272,17370,Vision Radiology,None,0,000002790287,112 Washington Pl Apt 17A,,Pittsburgh,PA,152193411,AM ...,None,2790287.0
3744,3943,Livingston Medical Group,None,0,000002086456,1140 Main St,,Livingston,CA,953341257,AM ...,2093941385,2086456.0
14994,16081,Cornerstone Family Healthcare XXX,None,0,000002536236,2570 9W Ste 10,,Cornwall,NY,12518,AM ...,None,2536236.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7665,7958,Alden Estates of Shorewood,None,1,000002101656,710 W Black Rd,,Shorewood,IL,604048400,LT ...,8152308700,2101656.0
7664,7957,David Martini MD Surgery Suite,None,1,000002101655,330 E Pulaski Hwy,,Elkton,MD,219216435,AM ...,4103986570,2101655.0
7663,7955,First GI Endoscopy & Surgery Center,None,1,000002101652,44 State Rt 23 Ste 1,,Riverdale,NJ,074571603,AM ...,9732481550,2101652.0
7638,7930,Sequel Pomegranate Health Systems,None,1,000002101626,765 Pierce Dr,,Columbus,OH,432232425,HP ...,6145195193,2101626.0


In [47]:
ALL.CUSTOMER_NBR = ALL.CUSTOMER_NBR.astype(float)
ALL_ADD = pd.merge(ALL, orga, on='CUSTOMER_NBR')

In [44]:
last_year = ppd[ppd.ME.isin(pd.unique(REAPP[REAPP.YR==2020].ME))]

In [45]:
last_year[last_year.POLO_MAILING_LINE_2.isna()]

,ME,MAILING_NAME,MAILING_LINE_1,MAILING_LINE_2,CITY,STATE,ZIP,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP
0,102000013,MICHAEL J STANFORD MD,NaN,4899 REYNOLDS LN,VESTAVIA,AL,35242,NaN,NaN,NaN,NaN,NaN
426,102021126,WENDY D KRIEGEL MD,NaN,566 LAKE CAMERON DR,PIKE ROAD,AL,36064,NaN,NaN,NaN,NaN,NaN
456,102021517,JEREMY B SMITH MD,JT 845 UAB DEPARTMENT OF ANESTHESIOLOGY,619 19TH ST S,BIRMINGHAM,AL,35233,NaN,NaN,NaN,NaN,NaN
772,102041453,DAVID T CHANDLER MD,NaN,28 MARITIME WAY,SANTA RSA BCH,FL,32459,NaN,NaN,NaN,NaN,NaN
785,102041585,ADAM S GREEN MD,DEPT OF RADIOLOGY,123 SUMMER ST,WORCESTER,MA,01608,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1280728,95701920075,ZORAN VUKCEVIC MD,NaN,234 N CENTRAL AVE STE 5000,PHOENIX,AZ,85004,NaN,NaN,NaN,NaN,NaN
1281078,95702780153,SLOBODAN ZEC MD,NaN,7324 HERITAGE CT APT 1D,FRANKFORT,IL,60423,NaN,NaN,NaN,NaN,NaN
1281241,95702910171,SVETLANA ANIC MD,NaN,PO BOX 2275,ATASCADERO,CA,93423,NaN,NaN,NaN,NaN,NaN
1281437,95704760010,ANETA K DIMOVA MD,NaN,3901 MICHAEL NEILL DR,AUSTIN,TX,78730,NaN,NaN,NaN,NaN,NaN


In [48]:
ALL_ADD

,FULL_DT,ME,PARTY_ID_x,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,YR,MONTH_NBR,CUSTOMER_NBR,...,PARTY_ID_y,STATUS,ADVANTAGE_ID,ADDRESS_LINE_1,ADDRESS_LINE_2,CITY,STATE,ZIPCODE,ORGANIZATION_TYPE,PHONE_NUMBER
0,2016-01-05,-1,-1,03418601,4915513,-1,1688,2016,1,2082967.0,...,None,1,000002082967,6100 Oak Tree Blvd,Ste 110,Independence,OH,44131,GR ...,8886385714
1,2017-11-09,03201940663,2181331,39325194,4915513,7200,1688,2017,11,2082967.0,...,None,1,000002082967,6100 Oak Tree Blvd,Ste 110,Independence,OH,44131,GR ...,8886385714
2,2017-05-12,03005841118,2181780,38687242,4915513,12091,1688,2017,5,2082967.0,...,None,1,000002082967,6100 Oak Tree Blvd,Ste 110,Independence,OH,44131,GR ...,8886385714
3,2017-06-12,03005841118,2181780,38823621,4915513,12091,1688,2017,6,2082967.0,...,None,1,000002082967,6100 Oak Tree Blvd,Ste 110,Independence,OH,44131,GR ...,8886385714
4,2016-05-18,03006870553,2184158,37076059,4915513,13703,1688,2016,5,2082967.0,...,None,1,000002082967,6100 Oak Tree Blvd,Ste 110,Independence,OH,44131,GR ...,8886385714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2233461,2021-02-01,49501790248,2733193,44068183,4915514,12387747,69950,2021,2,3468751.0,...,None,1,000003468751,1004 Pawlak Pkwy,,New Lenox,IL,604519401,HP ...,None
2233462,2021-02-02,49576810201,2743052,44069003,4915514,12394324,69950,2021,2,3468751.0,...,None,1,000003468751,1004 Pawlak Pkwy,,New Lenox,IL,604519401,HP ...,None
2233463,2021-02-09,49533790266,2789124,44081591,4915514,12440339,69950,2021,2,3468751.0,...,None,1,000003468751,1004 Pawlak Pkwy,,New Lenox,IL,604519401,HP ...,None
2233464,2021-01-19,30601050150,3477999,44033661,4915514,12699148,69950,2021,1,3468751.0,...,None,1,000003468751,1004 Pawlak Pkwy,,New Lenox,IL,604519401,HP ...,None


In [110]:
ppd[(ppd.ME.isin(pd.unique(REAPP.ME)))&(ppd.POLO_MAILING_LINE_2.isna())]

,ME,MAILING_NAME,MAILING_LINE_1,MAILING_LINE_2,CITY,STATE,ZIP,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP
0,00102000013,MICHAEL J STANFORD MD,NaN,4899 REYNOLDS LN,VESTAVIA,AL,35242,NaN,NaN,NaN,NaN,NaN
41,00102000455,TRACEY E BUTCHER MD,NaN,1090 HAMPTON PL,BIRMINGHAM,AL,35242,NaN,NaN,NaN,NaN,NaN
141,00102002211,AARON M GREENWALD MD,NaN,1 VA CTR STE 63E,AUGUSTA,ME,04330,1 VA MEDICAL CENTER STE 63E,NaN,AUGUSTA,ME,04330
214,00102010612,STEPHANIE M MORGAN MD,DOCTORS' CARE PC,307 E MEIGHAN BLVD,GADSDEN,AL,35903,NaN,NaN,NaN,NaN,NaN
296,00102011937,KEITH A WILLIAMSON MD,NaN,104 COVENTRY WYND RD,KINGSPORT,TN,37664,135 W RAVINE RD,NaN,KINGSPORT,TN,37660
...,...,...,...,...,...,...,...,...,...,...,...,...
1281583,95707830027,DRAGOSLAV M GVOZDJAN MD,NaN,10S641 GARFIELD AVE,BURR RIDGE,IL,60527,NaN,NaN,NaN,NaN,NaN
1281746,95708870227,AMIR BERBERKIC MD,NaN,5273 RFD,LONG GROVE,IL,60047,NaN,NaN,NaN,NaN,NaN
1281752,95708880079,SIMA SULER MD,NaN,12458 BIRTLE AVE,PT CHARLOTTE,FL,33981,NaN,NaN,NaN,NaN,NaN
1281781,95708910032,AGRON S ISMAILI MD,NaN,3260 E JAMES DR,OAK CREEK,WI,53154,NaN,NaN,NaN,NaN,NaN


In [51]:
641622/len(ppd)

0.5005046998712899

In [52]:
ALL.ME

0                  -1
1         03201940663
2         03005841118
3         03005841118
4         03006870553
             ...     
489806    30804080089
489807    00801930280
489808    00512140262
489809    05501060544
489810    01720140493
Name: ME, Length: 2234409, dtype: object

In [53]:
ppd['ME'] = fix_me(ppd.ME)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [118]:
all_ppd_2 = pd.merge(ALL_ADD,ppd, on='ME')

In [64]:
all_ppd = all_ppd[all_ppd.POLO_MAILING_LINE_2.isna()==False]

In [65]:
all_ppd.columns

Index(['FULL_DT', 'ME', 'PARTY_ID_x', 'ORDER_NBR', 'ORDER_PRODUCT_ID',
       'ORDER_PHYSICIAN_HIST_KEY', 'CUSTOMER_KEY', 'YR', 'MONTH_NBR',
       'CUSTOMER_NBR', 'CUSTOMER_ISELL_LOGIN', 'CUSTOMER_NAME',
       'CUSTOMER_TYPE_DESC', 'CUSTOMER_TYPE', 'CUSTOMER_CATEGORY_DESC',
       'ORG_ID', 'ORG_NAME', 'PARTY_ID_y', 'STATUS', 'ADVANTAGE_ID',
       'ADDRESS_LINE_1', 'ADDRESS_LINE_2', 'CITY_x', 'STATE_x', 'ZIPCODE',
       'ORGANIZATION_TYPE', 'PHONE_NUMBER', 'MAILING_NAME', 'MAILING_LINE_1',
       'MAILING_LINE_2', 'CITY_y', 'STATE_y', 'ZIP', 'POLO_MAILING_LINE_1',
       'POLO_MAILING_LINE_2', 'POLO_CITY', 'POLO_STATE', 'POLO_ZIP'],
      dtype='object')

In [66]:
keep_list = []
for row in all_ppd.itertuples():
    keep = False
    if row.STATE_x==row.POLO_STATE:
        keep = True
    keep_list.append(keep)

In [67]:
all_ppd['KEEP'] = keep_list

In [69]:
state_match = all_ppd[all_ppd.KEEP==True]

In [70]:
state_match[['POLO_ZIP','ZIPCODE']]

,POLO_ZIP,ZIPCODE
7,77030,77030
11,30236,317011960
13,30236,316031727
14,30236,312100887
17,33445,336341143
...,...,...
2215148,63119,631192951
2215149,13790,139024600
2215150,13903,139024600
2215151,8360,083606933


In [73]:
zip_matches = []
for row in state_match.itertuples():
    zip_match = False
    if str(row.POLO_ZIP) in str(row.ZIPCODE):
        zip_match = True
    zip_matches.append(zip_match)

In [74]:
state_match['ZIP_MATCH'] = zip_matches

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [75]:
state_match[state_match.ZIP_MATCH==True].drop_duplicates('ME')

,FULL_DT,ME,PARTY_ID_x,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,YR,MONTH_NBR,CUSTOMER_NBR,...,CITY_y,STATE_y,ZIP,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,KEEP,ZIP_MATCH
7,2019-06-07,03006870553,2184158,41517363,4915513,5402914,58289,2019,6,2320894.0,...,HOUSTON,TX,77030,DEPARTMENT OF RADIOLOGY,6431 FANNIN ST,HOUSTON,TX,77030,True,True
1468,2017-09-19,66104050043,3544243,39086535,4915513,1787302,638,2017,9,2080771.0,...,BELLBROOK,OH,45305,ST RITA'S MEDICAL CENTER,730 W MARKET ST,LIMA,OH,45801,True,True
1918,2016-10-28,91503980075,3374348,37545588,4915513,636989,4654,2016,10,2103603.0,...,BEAVER,PA,15009,NaN,4 PEARTREE WAY,BEAVER,PA,15009,True,True
2079,2019-03-05,05605952625,2621767,40715152,4915513,5548117,66727,2019,3,2081134.0,...,ELK GROVE VLG,IL,60007,NaN,800 BIESTERFIELD RD STE 610,ELK GROVE VLG,IL,60007,True,True
2324,2016-11-25,49503870010,2711563,37614251,4915514,342909,592,2016,11,2080743.0,...,NAPERVILLE,IL,60540,NaN,3825 HIGHLAND AVE,DOWNERS GROVE,IL,60515,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215120,2020-01-14,02401951167,2136014,42742621,4915514,7854191,131652,2020,1,3698116.0,...,BEVERLY HILLS,CA,90210,NaN,435 N BEDFORD DR STE 308,BEVERLY HILLS,CA,90210,True,True
2215136,2020-10-08,02401940653,2155745,43761764,4915514,9974549,126836,2020,10,3588926.0,...,TUCKER,GA,30084,NaN,1462 MONTREAL RD STE 101,TUCKER,GA,30084,True,True
2215143,2021-01-06,00102730968,1903949,43971723,4915514,11804307,142967,2021,1,4091328.0,...,LUVERNE,AL,36049,LUVERNE HEALTH CLINIC,39 ROY BEALL DR,LUVERNE,AL,36049,True,True
2215146,2021-02-05,24402820108,2627249,44075006,4915514,12532585,67093,2021,2,3411720.0,...,MUNSTER,IN,46321,EDGEWATER SYSTEMS FOR BALANCED LIVING,1100 W 6TH AVE,GARY,IN,46402,True,True


In [77]:
state_match[['POLO_MAILING_LINE_2','ADDRESS_LINE_1']]

,POLO_MAILING_LINE_2,ADDRESS_LINE_1
7,6431 FANNIN ST,6414 Fannin St
11,2400 MOUNT ZION PKWY,PO Box 3770
13,2400 MOUNT ZION PKWY,PO Box 1727
14,2400 MOUNT ZION PKWY,111 Perimeter Pkwy
17,225 S CONGRESS AVE,Ren 11st Floor
...,...,...
2215148,231 W LOCKWOOD AVE STE 202,231 W Lockwood Ave Ste 201
2215149,33-57 HARRISON ST,4400 Vestal Pkwy
2215150,STATE UNIV OF NEW YORK,4400 Vestal Pkwy
2215151,2950 COLLEGE DR STE 2E,2950 College Dr Ste 2E


In [210]:
from fuzzywuzzy import fuzz
def check_match(thing_1, thing_2):
    '''match check'''
    if thing_1 == thing_2:
        return True
    elif thing_1 in thing_2 or thing_2 in thing_1:
        return True
    elif fuzz.ratio(thing_1, thing_2) > 80:
        return True
    else:
        return False

In [80]:
adds = []
for row in state_match.itertuples():
    add_match =  check_match(row.POLO_MAILING_LINE_2, row.ADDRESS_LINE_1.upper())
    adds.append(add_match)

In [81]:
state_match['ADD_MATCH']=adds

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [86]:
state_match[(state_match.ADD_MATCH==True)|(state_match.ZIP_MATCH==True)].drop_duplicates('ME')[['POLO_MAILING_LINE_2','ADDRESS_LINE_1']]

,POLO_MAILING_LINE_2,ADDRESS_LINE_1
7,6431 FANNIN ST,6414 Fannin St
107,1 HOSPITAL DR,1 Hospital Dr
226,835 HOSPITAL RD,1 Hospital Dr
245,2500 METROHEALTH DR,2500 Metrohealth Dr # G731-C
675,2001 MEDICAL PKWY,2001 Medical Pkwy
...,...,...
2215142,233 E SHORE RD,233 E Shore Rd Ste 101
2215143,39 ROY BEALL DR,150 Hospital Drive
2215146,1100 W 6TH AVE,1100 W 6th Ave
2215148,231 W LOCKWOOD AVE STE 202,231 W Lockwood Ave Ste 201


In [88]:
len(all_ppd.drop_duplicates('ME'))

524567

In [91]:
len(state_match.drop_duplicates('ME'))

421798

array([2320894., 2080771., 2079965., ..., 3588926., 3411720., 2081369.])

In [94]:
len(orga)

17219

In [102]:
state_match[(state_match.ADD_MATCH==True)&(state_match.ZIP_MATCH==True)&(state_match.CUSTOMER_CATEGORY_DESC=='Credit Verification Org')][['CUSTOMER_NAME','ADDRESS_LINE_1','POLO_MAILING_LINE_2']]

,CUSTOMER_NAME,ADDRESS_LINE_1,POLO_MAILING_LINE_2
16957,Apogee Physicians,8117 Preston Rd Ste 800,8117 PRESTON RD STE 800
26918,Community Healthcare System Cvo,1500 S Lake Park Ave,1500 S LAKE PARK AVE STE 110
63307,N-Query Credentialing Services,7251 W 4th St,2410 W 16TH ST
214358,Apogee Physicians,8117 Preston Rd Ste 800,8117 PRESTON RD STE 800
238821,Credentia Source,1111 6th Ave,1111 6TH AVE FL 2
...,...,...,...
2129781,Community Healthcare System Cvo,1500 S Lake Park Ave,1400 S LAKE PARK AVE STE 202
2129795,Community Healthcare System Cvo,1500 S Lake Park Ave,1400 S LAKE PARK AVE STE 205
2129799,Community Healthcare System Cvo,1500 S Lake Park Ave,1400 S LAKE PARK AVE STE 105
2129800,Community Healthcare System Cvo,1500 S Lake Park Ave,1400 S LAKE PARK AVE STE 105


In [132]:
state_match[((state_match.ADD_MATCH==True)|(state_match.ZIP_MATCH==True))&(state_match.YR==2020)].drop_duplicates('ME').groupby('MONTH_NBR').count()[['FULL_DT']]

,FULL_DT
MONTH_NBR,
1,1537
2,1451
3,1708
4,1417
5,1438
6,1477
7,1467
8,1305
9,1177


In [116]:
good_customers = pd.unique(state_match[(state_match.ADD_MATCH==True)&(state_match.ZIP_MATCH==True)].CUSTOMER_NBR)

In [117]:
all_ppd

,FULL_DT,ME,PARTY_ID_x,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,YR,MONTH_NBR,CUSTOMER_NBR,...,MAILING_LINE_2,CITY_y,STATE_y,ZIP,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,KEEP
2,2017-05-12,03005841118,2181780,38687242,4915513,12091,1688,2017,5,2082967.0,...,406 S 30TH AVE STE 101,YAKIMA,WA,98902,NaN,406 S 30TH AVE STE 101,YAKIMA,WA,98902,False
3,2017-06-12,03005841118,2181780,38823621,4915513,12091,1688,2017,6,2082967.0,...,406 S 30TH AVE STE 101,YAKIMA,WA,98902,NaN,406 S 30TH AVE STE 101,YAKIMA,WA,98902,False
4,2016-06-28,03005841118,2181780,37206399,4915513,12091,641,2016,6,2080778.0,...,406 S 30TH AVE STE 101,YAKIMA,WA,98902,NaN,406 S 30TH AVE STE 101,YAKIMA,WA,98902,False
5,2017-06-23,03005841118,2181780,38846408,4915513,12091,5972,2017,6,2100340.0,...,406 S 30TH AVE STE 101,YAKIMA,WA,98902,NaN,406 S 30TH AVE STE 101,YAKIMA,WA,98902,False
6,2016-05-18,03006870553,2184158,37076059,4915513,13703,1688,2016,5,2082967.0,...,6431 FANNIN ST,HOUSTON,TX,77030,DEPARTMENT OF RADIOLOGY,6431 FANNIN ST,HOUSTON,TX,77030,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215148,2021-02-02,02802931355,2252324,44069993,4915514,12046302,148037,2021,2,2081369.0,...,231 W LOCKWOOD AVE STE 201,WEBSTER GRVS,MO,63119,NaN,231 W LOCKWOOD AVE STE 202,SAINT LOUIS,MO,63119,True
2215149,2021-02-02,03575901084,2216164,44069150,4915514,12166537,148040,2021,2,2108682.0,...,3 CAMILLA ST,CONKLIN,NY,13748,UPSTATE MEDICAL UNIV,33-57 HARRISON ST,JOHNSON CITY,NY,13790,True
2215150,2021-02-02,65401910012,2734424,44069150,4915514,12412748,148040,2021,2,2108682.0,...,4182 MURPHY RD,BINGHAMTON,NY,13903,HEALTH SERVICES,STATE UNIV OF NEW YORK,BINGHAMTON,NY,13903,True
2215151,2021-02-10,03305960569,2327852,44085730,4915514,12195635,57282,2021,2,2105525.0,...,4 WOODGLEN LN,VOORHEES,NJ,8043,NaN,2950 COLLEGE DR STE 2E,VINELAND,NJ,8360,True


In [131]:
all_ppd_2[(all_ppd_2.ME.isin(ALL.ME))&(all_ppd_2.POLO_MAILING_LINE_2.isna())&(all_ppd_2.POLO_MAILING_LINE_1.isna())&(all_ppd_2.YR==2020)].drop_duplicates('ME').groupby('MONTH_NBR').count()[['FULL_DT']]

,FULL_DT
MONTH_NBR,
1,1363
2,1275
3,1421
4,1158
5,1134
6,1124
7,1078
8,1024
9,1061


In [139]:
these = usages[usages.PARTY_ID.isin(ALL_ADD.PARTY_ID_x)]

In [135]:
ALL_ADD

,FULL_DT,ME,PARTY_ID_x,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,YR,MONTH_NBR,CUSTOMER_NBR,...,PARTY_ID_y,STATUS,ADVANTAGE_ID,ADDRESS_LINE_1,ADDRESS_LINE_2,CITY,STATE,ZIPCODE,ORGANIZATION_TYPE,PHONE_NUMBER
0,2016-01-05,-1,-1,03418601,4915513,-1,1688,2016,1,2082967.0,...,None,1,000002082967,6100 Oak Tree Blvd,Ste 110,Independence,OH,44131,GR ...,8886385714
1,2017-11-09,03201940663,2181331,39325194,4915513,7200,1688,2017,11,2082967.0,...,None,1,000002082967,6100 Oak Tree Blvd,Ste 110,Independence,OH,44131,GR ...,8886385714
2,2017-05-12,03005841118,2181780,38687242,4915513,12091,1688,2017,5,2082967.0,...,None,1,000002082967,6100 Oak Tree Blvd,Ste 110,Independence,OH,44131,GR ...,8886385714
3,2017-06-12,03005841118,2181780,38823621,4915513,12091,1688,2017,6,2082967.0,...,None,1,000002082967,6100 Oak Tree Blvd,Ste 110,Independence,OH,44131,GR ...,8886385714
4,2016-05-18,03006870553,2184158,37076059,4915513,13703,1688,2016,5,2082967.0,...,None,1,000002082967,6100 Oak Tree Blvd,Ste 110,Independence,OH,44131,GR ...,8886385714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2233461,2021-02-01,49501790248,2733193,44068183,4915514,12387747,69950,2021,2,3468751.0,...,None,1,000003468751,1004 Pawlak Pkwy,,New Lenox,IL,604519401,HP ...,None
2233462,2021-02-02,49576810201,2743052,44069003,4915514,12394324,69950,2021,2,3468751.0,...,None,1,000003468751,1004 Pawlak Pkwy,,New Lenox,IL,604519401,HP ...,None
2233463,2021-02-09,49533790266,2789124,44081591,4915514,12440339,69950,2021,2,3468751.0,...,None,1,000003468751,1004 Pawlak Pkwy,,New Lenox,IL,604519401,HP ...,None
2233464,2021-01-19,30601050150,3477999,44033661,4915514,12699148,69950,2021,1,3468751.0,...,None,1,000003468751,1004 Pawlak Pkwy,,New Lenox,IL,604519401,HP ...,None


In [136]:
all_ppd[all_ppd.KEEP==False]

,FULL_DT,ME,PARTY_ID_x,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,YR,MONTH_NBR,CUSTOMER_NBR,...,MAILING_LINE_2,CITY_y,STATE_y,ZIP,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,KEEP
2,2017-05-12,03005841118,2181780,38687242,4915513,12091,1688,2017,5,2082967.0,...,406 S 30TH AVE STE 101,YAKIMA,WA,98902,NaN,406 S 30TH AVE STE 101,YAKIMA,WA,98902,False
3,2017-06-12,03005841118,2181780,38823621,4915513,12091,1688,2017,6,2082967.0,...,406 S 30TH AVE STE 101,YAKIMA,WA,98902,NaN,406 S 30TH AVE STE 101,YAKIMA,WA,98902,False
4,2016-06-28,03005841118,2181780,37206399,4915513,12091,641,2016,6,2080778.0,...,406 S 30TH AVE STE 101,YAKIMA,WA,98902,NaN,406 S 30TH AVE STE 101,YAKIMA,WA,98902,False
5,2017-06-23,03005841118,2181780,38846408,4915513,12091,5972,2017,6,2100340.0,...,406 S 30TH AVE STE 101,YAKIMA,WA,98902,NaN,406 S 30TH AVE STE 101,YAKIMA,WA,98902,False
6,2016-05-18,03006870553,2184158,37076059,4915513,13703,1688,2016,5,2082967.0,...,6431 FANNIN ST,HOUSTON,TX,77030,DEPARTMENT OF RADIOLOGY,6431 FANNIN ST,HOUSTON,TX,77030,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215023,2021-02-17,52801860087,2731857,44098817,4915514,12396962,69724,2021,2,3218067.0,...,1312 PROFESSIONAL BLVD STE 201,EVANSVILLE,IN,47714,NaN,1312 PROFESSIONAL BLVD STE 201,EVANSVILLE,IN,47714,False
2215068,2018-11-15,01902791121,1966683,40403785,4915514,4310280,56457,2018,11,3087978.0,...,1100 HIGHLAND DR,CONCORDIA,KS,66901,NaN,515 WASHINGTON ST,CONCORDIA,KS,66901,False
2215069,2020-04-22,01902791121,1966683,43218331,4915514,9265123,56457,2020,4,3087978.0,...,1100 HIGHLAND DR,CONCORDIA,KS,66901,NaN,515 WASHINGTON ST,CONCORDIA,KS,66901,False
2215106,2020-07-16,03806900523,2366345,43451705,4915514,10138789,58051,2020,7,2947441.0,...,1438 S GRAND BLVD,SAINT LOUIS,MO,63104,DEPARTMENT OF PSYCHIATRY,1438 S GRAND BLVD,SAINT LOUIS,MO,63104,False


In [140]:
these['FROM_DT']>dt.date(2019,6,29)

,PARTY_ID,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_CD,SRC_POST_KEY,ZIP,FROM_DT,POST_KEY,PURPOSE_TYPE_DESC,PURPOSE_TYPE_ID,PURPOSE_USG_DESC
23,1798757,15014282,2 Sweetwater Dr,,,Cataula,GA,719824.0,31804,2005-12-16 20:20:34.882601,719824.0,Home,269,Preferred Professional Mailing Address
27,1888752,16182617,6621 Kathmoor Dr,,,Montgomery,AL,720015.0,36117,2005-12-16 20:20:34.887081,720015.0,Home,269,Preferred Professional Mailing Address
31,1845004,14894703,303 Smith St,,,Lagrange,GA,720308.0,30240,2005-12-16 20:20:34.895541,720308.0,Office,317,Preferred Office
32,1798496,16188043,2055 Normandie Dr,,,Montgomery,AL,720327.0,36111,2005-12-16 20:20:34.896344,720327.0,Office,318,Preferred Professional Mailing Address
34,1888761,16091619,3603 Cliff Rd S,,,Birmingham,AL,720419.0,35205,2005-12-16 20:20:34.899560,720419.0,Home,269,Preferred Professional Mailing Address
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13743101,3508392,32070545,145 King Of Prussia Rd,Ste 310 South,,Radnor,PA,25700040.0,19087,2021-02-21 20:45:40.457095,25700040.0,Office,318,Preferred Professional Mailing Address
13743102,5886493,32070434,201 E Huron St,Galter 11-140,,Chicago,IL,25700043.0,60611,2021-02-21 20:45:40.457224,25700043.0,Office,468,No Usage
13743103,5886493,32070434,201 E Huron St,Galter 11-140,,Chicago,IL,25700043.0,60611,2021-02-21 20:45:40.459205,25700043.0,Office,547,Membership Shipping. ...
13743104,5886493,32070482,2530 Erwin Rd,Apt 442,,Durham,NC,25700049.0,27705,2021-02-21 20:45:40.459331,25700049.0,Home,405,No Usage


In [145]:
perfect_match = pd.merge(these, ALL_ADD, left_on=['PARTY_ID','ADDR_1'], right_on=['PARTY_ID_x', 'ADDRESS_LINE_1'])

In [146]:
zip_match = pd.merge(these, ALL_ADD, left_on=['PARTY_ID','ZIP'], right_on=['PARTY_ID_x', 'ZIPCODE'])

In [142]:
ALL_ADD

,FULL_DT,ME,PARTY_ID_x,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,YR,MONTH_NBR,CUSTOMER_NBR,...,PARTY_ID_y,STATUS,ADVANTAGE_ID,ADDRESS_LINE_1,ADDRESS_LINE_2,CITY,STATE,ZIPCODE,ORGANIZATION_TYPE,PHONE_NUMBER
0,2016-01-05,-1,-1,03418601,4915513,-1,1688,2016,1,2082967.0,...,None,1,000002082967,6100 Oak Tree Blvd,Ste 110,Independence,OH,44131,GR ...,8886385714
1,2017-11-09,03201940663,2181331,39325194,4915513,7200,1688,2017,11,2082967.0,...,None,1,000002082967,6100 Oak Tree Blvd,Ste 110,Independence,OH,44131,GR ...,8886385714
2,2017-05-12,03005841118,2181780,38687242,4915513,12091,1688,2017,5,2082967.0,...,None,1,000002082967,6100 Oak Tree Blvd,Ste 110,Independence,OH,44131,GR ...,8886385714
3,2017-06-12,03005841118,2181780,38823621,4915513,12091,1688,2017,6,2082967.0,...,None,1,000002082967,6100 Oak Tree Blvd,Ste 110,Independence,OH,44131,GR ...,8886385714
4,2016-05-18,03006870553,2184158,37076059,4915513,13703,1688,2016,5,2082967.0,...,None,1,000002082967,6100 Oak Tree Blvd,Ste 110,Independence,OH,44131,GR ...,8886385714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2233461,2021-02-01,49501790248,2733193,44068183,4915514,12387747,69950,2021,2,3468751.0,...,None,1,000003468751,1004 Pawlak Pkwy,,New Lenox,IL,604519401,HP ...,None
2233462,2021-02-02,49576810201,2743052,44069003,4915514,12394324,69950,2021,2,3468751.0,...,None,1,000003468751,1004 Pawlak Pkwy,,New Lenox,IL,604519401,HP ...,None
2233463,2021-02-09,49533790266,2789124,44081591,4915514,12440339,69950,2021,2,3468751.0,...,None,1,000003468751,1004 Pawlak Pkwy,,New Lenox,IL,604519401,HP ...,None
2233464,2021-01-19,30601050150,3477999,44033661,4915514,12699148,69950,2021,1,3468751.0,...,None,1,000003468751,1004 Pawlak Pkwy,,New Lenox,IL,604519401,HP ...,None


In [149]:
easy_match = pd.concat([perfect_match, zip_match]).drop_duplicates()

In [165]:
easy_match['DATE_DIFFERENCE'] = easy_match.FROM_DT - easy_match.FULL_DT

In [159]:
easy_match['FULL_DT']=pd.to_datetime(easy_match.FULL_DT)

In [176]:
BOP = easy_match[easy_match.DATE_DIFFERENCE>dt.timedelta(0)].sort_values('DATE_DIFFERENCE').drop_duplicates(['PARTY_ID','CUSTOMER_NBR'])

In [168]:
import datetime as dt

In [172]:
dt.timedelta(0)

datetime.timedelta(0)

In [178]:
BOP['Day_Difference'] = [x.days for x in list(BOP['DATE_DIFFERENCE'])]

In [182]:
len(BOP)

58438

In [187]:
new_easy = easy_match[easy_match.ORDER_PRODUCT_ID==4915513]

In [181]:
dayz = list(BOP[BOP['Day_Difference']>0]['Day_Difference'])

import matplotlib.pyplot as plt
import numpy as np
# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(x=dayz, bins=15, color='#0504aa',
                            alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Days')
plt.ylabel('Frequency')
plt.title('Amount of Days until Address Appears in Record')
plt.text(23, 45, r'$\mu=15, b=3$')
maxfreq = n.max()
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)

(0.0, 18880.0)

In [189]:
boop = new_easy[new_easy.DATE_DIFFERENCE>dt.timedelta(0)].sort_values('DATE_DIFFERENCE').drop_duplicates(['PARTY_ID','CUSTOMER_NBR'])

In [193]:
BOP[(BOP['Day_Difference']>0)&(BOP['Day_Difference']<=92)]

,PARTY_ID,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY_x,STATE_CD,SRC_POST_KEY,ZIP,FROM_DT,...,ADVANTAGE_ID,ADDRESS_LINE_1,ADDRESS_LINE_2,CITY_y,STATE,ZIPCODE,ORGANIZATION_TYPE,PHONE_NUMBER,DATE_DIFFERENCE,Day_Difference
180599,2485628,23381306,7026 Old Katy Rd Ste 276,,,Houston,TX,18833307.0,77024,2017-07-06 00:06:45.000000,...,000002101734,7026 Old Katy Rd Ste 276,,Houston,TX,770242187,GR ...,7133580562,1 days 00:06:45,1
17807,6398105,29238144,6431 Fannin St Ste 3.262,None,None,Houston,TX,22850306.0,77030,2020-07-11 00:13:08.000000,...,000002320894,6414 Fannin St,Ste G125,Houston,TX,77030,MGC ...,7135006932,1 days 00:13:08,1
180081,2213436,15193226,163 Hospital Dr,Stephens County Hospital,,Toccoa,GA,8177087.0,30577,2017-06-21 00:15:10.000000,...,000002086884,163 Hospital Dr,,Toccoa,GA,305779700,HP ...,7062824276,1 days 00:15:10,1
183966,5837020,22768422,1 Childrens Plz,,,Dayton,OH,18154064.0,45404,2017-08-03 00:18:53.000000,...,000002084487,1 Childrens Plz,,Dayton,OH,454041815,HP ...,9376413456,1 days 00:18:53,1
178788,3357624,12469506,1324 Lakeland Hills Blvd,,,Lakeland,FL,8312519.0,33805,2017-05-31 00:23:44.000000,...,000002080324,1324 Lakeland Hills Blvd,,Lakeland,FL,338054500,HP ...,8636871053,1 days 00:23:44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,2861339,27421681,4502 Medical Dr,University Hospital Emergency Department,None,San Antonio,TX,21342073.0,78229,2017-03-17 21:55:47.000000,...,000002080143,8431 Fredericksburg Road,,San Antonio,TX,78229,GR ...,2104504873,92 days 21:55:47,92
118290,2764457,22645218,172 Kinsley St,St Joseph Hospital,,Nashua,NH,18029871.0,03060,2016-09-15 22:35:43.463664,...,000002080257,172 Kinsley St,,Nashua,NH,030603688,HP ...,6035953032,92 days 22:35:43.463664,92
175638,4536149,27461706,4800 Sand Point Way NE,Oa.9.120.1 - Orthopedics Administration,None,Seattle,WA,21370637.0,98105,2017-03-17 22:40:52.000000,...,000002081839,4800 Sand Point Way NE,,Seattle,WA,981053916,HP ...,2069875219,92 days 22:40:52,92
3017,2706887,27428145,17 Hospital Hill Rd,P O Box 1736,None,Sharon,CT,21346771.0,06069,2017-03-17 22:40:52.000000,...,000002086295,50 Hospital Hill Rd,PO Box 789,Sharon,CT,06069,HP ...,8603644008,92 days 22:40:52,92


In [198]:
year_2 = ALL_ADD[ALL_ADD.FULL_DT>dt.date(2019,2,22)]

In [200]:
yearo = these[these['FROM_DT']>dt.date(2019,2,22)]

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


In [206]:
year_2.columns

Index(['FULL_DT', 'ME', 'PARTY_ID_x', 'ORDER_NBR', 'ORDER_PRODUCT_ID',
       'ORDER_PHYSICIAN_HIST_KEY', 'CUSTOMER_KEY', 'YR', 'MONTH_NBR',
       'CUSTOMER_NBR', 'CUSTOMER_ISELL_LOGIN', 'CUSTOMER_NAME',
       'CUSTOMER_TYPE_DESC', 'CUSTOMER_TYPE', 'CUSTOMER_CATEGORY_DESC',
       'ORG_ID', 'ORG_NAME', 'PARTY_ID_y', 'STATUS', 'ADVANTAGE_ID',
       'ADDRESS_LINE_1', 'ADDRESS_LINE_2', 'CITY', 'STATE', 'ZIPCODE',
       'ORGANIZATION_TYPE', 'PHONE_NUMBER'],
      dtype='object')

In [204]:
year_2.groupby('ORDER_PRODUCT_ID').count()

,FULL_DT,ME,PARTY_ID_x,ORDER_NBR,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,YR,MONTH_NBR,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,...,PARTY_ID_y,STATUS,ADVANTAGE_ID,ADDRESS_LINE_1,ADDRESS_LINE_2,CITY,STATE,ZIPCODE,ORGANIZATION_TYPE,PHONE_NUMBER
ORDER_PRODUCT_ID,,,,,,,,,,,,,,,,,,,,,
4915513,683039,683039,683039,683039,683039,683039,683039,683039,683039,683039,...,0,683039,683039,683039,683039,683039,683039,683039,683039,618469
4915514,196221,196221,196221,196221,196221,196221,196221,196221,196221,196221,...,0,196221,196221,196221,196221,196221,196221,196221,196221,177307


In [208]:
attempt = pd.merge(yearo, year_2, left_on=['PARTY_ID','STATE_CD'], right_on=['PARTY_ID_x', 'STATE'])

In [211]:
OY_LIST = []
for row in attempt.itertuples():
    OY = check_match(row.ADDR_1, row.ADDRESS_LINE_1)
    OY_LIST.append(OY)

In [212]:
attempt['ADDRESS_MATCH']=OY_LIST

In [213]:
attempt[attempt['ADDRESS_MATCH']==True].

,PARTY_ID,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY_x,STATE_CD,SRC_POST_KEY,ZIP,FROM_DT,...,STATUS,ADVANTAGE_ID,ADDRESS_LINE_1,ADDRESS_LINE_2,CITY_y,STATE,ZIPCODE,ORGANIZATION_TYPE,PHONE_NUMBER,ADDRESS_MATCH
0,2187335,9357196,1617 N California St,Ste 2D,,Stockton,CA,3312458.0,95204,2019-02-22 00:27:45.000000,...,1,000002080260,1800 N California St,,Stockton,CA,952046019,HP ...,2094676556,True
1,2187335,9357196,1617 N California St,Ste 2D,,Stockton,CA,3312458.0,95204,2019-12-07 02:31:18.000000,...,1,000002080260,1800 N California St,,Stockton,CA,952046019,HP ...,2094676556,True
2,2187335,9357196,1617 N California St,Ste 2D,,Stockton,CA,3312458.0,95204,2020-04-05 11:28:55.000000,...,1,000002080260,1800 N California St,,Stockton,CA,952046019,HP ...,2094676556,True
72,6137557,28934769,2201 Lexington Ave,,,Ashland,KY,22479669.0,41101,2019-04-08 21:20:36.152050,...,1,000002080813,2201 Lexington Ave,,Ashland,KY,411012843,HP ...,6064080413,True
73,6137557,28934769,2201 Lexington Ave,,,Ashland,KY,22479669.0,41101,2019-04-08 21:20:36.152050,...,1,000002080813,2201 Lexington Ave,,Ashland,KY,411012843,HP ...,6064080413,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535030,6825809,11589101,3401 Civic Center Blvd,,,Philadelphia,PA,6807146.0,19104,2021-02-21 20:44:56.631312,...,1,000002081799,3401 Civic Center Blvd,,Philadelphia,PA,191044319,HP ...,2155901994,True
1535040,3736677,16996783,1717 S Calhoun St,,,Fort Wayne,IN,8097362.0,46802,2021-02-21 20:44:59.518212,...,1,000002101525,1717 S Calhoun St,,Fort Wayne,IN,468025257,AM ...,2604582641,True
1535041,3736677,16996783,1717 S Calhoun St,,,Fort Wayne,IN,8097362.0,46802,2021-02-21 20:44:59.526469,...,1,000002101525,1717 S Calhoun St,,Fort Wayne,IN,468025257,AM ...,2604582641,True
1535120,2534307,32070402,,220 Canterbury Drive,,Hays,KS,25699595.0,67601,2021-02-21 20:45:38.078357,...,1,000002083707,1700 SW 7th St,,Topeka,KS,666061690,HP ...,7852958887,True


In [24]:
pd.read_csv('../../Data/MasterfileCore/STUDENT_SAMPLE.TXT', header=None, delimiter='|')[[26]]

,26
0,3.0
1,3.0
2,1.0
3,3.0
4,2.0
...,...
95,2.0
96,1.0
97,1.0
98,3.0
